In [4]:
#!/usr/bin/env python3
"""
=============================================================================
INTEGRATED ECONOMIC ANALYSIS - AMMONIA PLANT (PLASMA-BASED)
WITH DYNAMIC EQUIPMENT COSTING AND ALL MATERIAL INPUTS FROM STREAM DATA
=============================================================================

This module integrates:
1. Equipment cost estimation using thesis-validated correlations
2. Complete techno-economic analysis with all material inputs

The equipment costs are calculated dynamically using validated correlations
rather than hardcoded values, ensuring consistency and enabling sensitivity
analysis on equipment sizing and technology parameters.

INPUT MATERIALS:
- Stream 13: Demineralized water to PEM electrolyzer (50 kgmol/h)
- Stream 27: Nitrogen feed from ASU (69 kgmol/h)
- Stream 55: Air for combustion (free)
- Stream 1: Geothermal steam (energy source, not consumed)

OUTPUT PRODUCTS:
- Streams 46+49: Liquid ammonia product (30.62 kgmol/h total)
- Stream 25: Oxygen byproduct from PEM (25 kgmol/h)
- Electricity: Net export from turbines
- District heating: Hot water

References:
- Lei et al. (2024), Energy 307, 132728
- Peters, Timmerhaus, West (2003)
- Turton et al. (2018)
- DOE H2NEW Consortium (2024). NREL/TP-5900-87625
- Delikonstantis et al. (2017), Martini et al. (2019) - DBD Plasma Reactor
"""

import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from typing import Dict, Tuple, Optional
import warnings

# =============================================================================
# SECTION 1: CEPCI DATA AND ADJUSTMENT FUNCTIONS
# =============================================================================

CEPCI_DATA = {
    1995: 381.1, 1996: 381.7, 1997: 386.5, 1998: 389.5, 1999: 390.6,
    2000: 394.1, 2001: 394.3, 2002: 395.6, 2003: 402.0, 2004: 444.2,
    2005: 468.2, 2006: 499.6, 2007: 525.4, 2008: 575.4, 2009: 521.9,
    2010: 550.8, 2011: 585.7, 2012: 584.6, 2013: 567.3, 2014: 576.1,
    2015: 556.8, 2016: 541.7, 2017: 567.5, 2018: 603.1, 2019: 607.5,
    2020: 596.2, 2021: 708.0, 2022: 816.0, 2023: 797.0, 2024: 800.0, 
    2025: 834.0
}

TARGET_YEAR = 2025
CEPCI_TARGET = CEPCI_DATA[TARGET_YEAR]
CEPCI_BASE = CEPCI_DATA[2017]  # Turton base year


def cepci_adjust(cost_base: float, year_base: int = 2017) -> float:
    """
    Adjust cost to target year using CEPCI (Chemical Engineering Plant Cost Index).
    
    The CEPCI is the industry standard for scaling equipment costs across different
    years to account for inflation and market changes in the chemical industry.
    """
    if year_base not in CEPCI_DATA:
        return cost_base
    return cost_base * (CEPCI_TARGET / CEPCI_DATA[year_base])


# =============================================================================
# SECTION 2: EQUIPMENT COST FUNCTIONS
# =============================================================================

def calc_control_valve(Cv: float) -> Dict:
    """
    Control valve cost using Turton (2018) Table A.1.
    
    Parameters:
    -----------
    Cv : float
        Valve flow coefficient (gpm at ΔP = 1 psi)
    """
    Cv = max(0.5, Cv)
    K1, K2, K3 = 3.7119, 0.5974, 0.0
    ln_Cv = np.log(Cv)
    Cp = np.exp(K1 + K2 * ln_Cv + K3 * ln_Cv**2)
    Cp_adj = cepci_adjust(Cp, 2017)
    Cp_adj = max(500, Cp_adj)
    F_BM = 1.3
    C_BM = Cp_adj * F_BM
    
    return {
        'purchased': round(Cp_adj, 0),
        'installed': round(C_BM, 0),
        'method': 'Turton (2018) Table A.1'
    }


def calc_vessel(volume_m3: float, diameter_m: float, P_barg: float = 10.0,
                orientation: str = 'vertical') -> Dict:
    """
    Pressure vessel cost using Peters-Timmerhaus power law correlation.
    
    Reference: Peters, Timmerhaus, West (2003). Plant Design and Economics.
    """
    height_m = volume_m3 / (np.pi * (diameter_m/2)**2)
    D_ft = diameter_m * 3.281
    L_ft = height_m * 3.281
    D_in = diameter_m * 39.37
    P_psig = max(15, P_barg * 14.5038)
    
    S = 15000
    E = 0.85
    t_shell = (P_psig * D_in) / (2 * S * E - 1.2 * P_psig)
    t_shell = max(0.25, min(3.0, t_shell))
    
    rho_steel = 490
    W_cyl = np.pi * D_ft * L_ft * (t_shell/12) * rho_steel
    W_heads = 2 * 0.25 * np.pi * (D_ft**2) * (t_shell/12) * rho_steel
    W_lb = max(1000, W_cyl + W_heads)
    
    a, n = 600, 0.65
    Cp_2003 = a * (W_lb ** n)
    CEPCI_2003 = 402.0
    Cp_adj = Cp_2003 * (CEPCI_TARGET / CEPCI_2003)
    
    Fp = 1.0 if P_barg <= 10 else min(2.0, 1.0 + 0.1 * (P_barg - 10) / 10)
    Cp_adj = Cp_adj * Fp
    
    F_BM = 4.07 if orientation == 'vertical' else 3.05
    C_BM = Cp_adj * F_BM
    
    return {
        'purchased': round(Cp_adj, 0),
        'installed': round(C_BM, 0),
        'W_lb': round(W_lb, 0),
        't_in': round(t_shell, 3),
        'Fp': round(Fp, 2),
        'F_BM': round(F_BM, 2),
        'method': 'Peters-Timmerhaus (2003) power law'
    }


def calc_turbine(power_kW: float) -> Dict:
    """
    Turbine cost using weighted average of Turton (2018) and Bejan (1996).
    """
    power_kW = max(10, power_kW)
    power_hp = power_kW * 1.341
    
    K1, K2, K3 = 2.7051, 1.4398, -0.1776
    power_hp_calc = max(100, min(4000, power_hp))
    ln_P = np.log(power_hp_calc)
    Cp_turton = np.exp(K1 + K2 * ln_P + K3 * ln_P**2)
    
    if power_hp > 4000:
        Cp_turton = Cp_turton * (power_hp / 4000) ** 0.6
    
    Cp_turton_adj = cepci_adjust(Cp_turton, 2017)
    
    Cp_bejan = 6000 * (power_kW ** 0.7)
    Cp_bejan_adj = cepci_adjust(Cp_bejan, 1995)
    
    Cp_avg = 0.6 * Cp_turton_adj + 0.4 * Cp_bejan_adj
    F_BM = 3.5
    C_BM = Cp_avg * F_BM
    
    return {
        'purchased': round(Cp_avg, 0),
        'installed': round(C_BM, 0),
        'cost_per_kW': round(Cp_avg / power_kW, 0),
        'method': 'Turton (2018) + Bejan (1996)'
    }


def calc_compressor(power_kW: float, comp_type: str = 'centrifugal') -> Dict:
    """
    Compressor cost using Turton (2018) Table A.1.
    """
    power_kW = max(5, power_kW)
    power_hp = power_kW * 1.341
    
    K1, K2, K3 = 2.2897, 1.3604, -0.1027
    power_hp_calc = max(450, min(3000, power_hp))
    ln_P = np.log(power_hp_calc)
    Cp = np.exp(K1 + K2 * ln_P + K3 * ln_P**2)
    
    if power_hp < 450:
        Cp = Cp * (power_hp / 450) ** 0.6
    elif power_hp > 3000:
        Cp = Cp * (power_hp / 3000) ** 0.6
    
    Cp_adj = cepci_adjust(Cp, 2017)
    Cp_adj = max(50000, Cp_adj)
    
    F_BM = 2.15
    C_BM = Cp_adj * F_BM
    
    return {
        'purchased': round(Cp_adj, 0),
        'installed': round(C_BM, 0),
        'cost_per_kW': round(Cp_adj / power_kW, 0),
        'method': 'Turton (2018) Table A.1'
    }


def calc_pump(power_kW: float, head_m: float = 100.0, flow_m3h: float = 100.0) -> Dict:
    """
    Pump + motor cost using industry-validated power law correlation.
    
    Reference: Industry data from Pumps & Systems (Almasi, 2020)
    """
    power_kW = max(1, power_kW)
    
    a, b = 1000, 0.70
    Cp_2020 = a * (power_kW ** b)
    Cp_2020 = max(5000, Cp_2020)
    
    CEPCI_2020 = 596.2
    Cp_adj = Cp_2020 * (CEPCI_TARGET / CEPCI_2020)
    
    if head_m > 100:
        Fp = min(1.5, 1.0 + 0.005 * (head_m - 100))
        Cp_adj = Cp_adj * Fp
    
    F_BM = 2.4
    C_BM = Cp_adj * F_BM
    
    return {
        'purchased': round(Cp_adj, 0),
        'installed': round(C_BM, 0),
        'cost_per_kW': round(Cp_adj / power_kW, 0),
        'method': 'Power law (Almasi 2020 industry data)'
    }


def calc_heat_exchanger(area_m2: float, hx_type: str = 'shell_tube',
                         P_barg: float = 10.0) -> Dict:
    """
    Heat exchanger cost using industry-validated power law correlation.
    
    References:
    - Thunder Said Energy (2023): $100-500/m² for industrial HX
    - Xylem Heat Transfer (2024): $1M-$10M for large shell & tube
    """
    MAX_AREA = 1000  # m² per shell (TEMA guideline)
    
    if np.isnan(area_m2) or area_m2 <= 0:
        area_m2 = 10
    
    if 'plate' in hx_type.lower():
        base_cost_per_m2, scale_exp = 300, 0.85
    elif 'air' in hx_type.lower() or 'finned' in hx_type.lower() or 'whru' in hx_type.lower():
        base_cost_per_m2, scale_exp = 250, 0.80
    elif 'cryo' in hx_type.lower() or 'aluminum' in hx_type.lower():
        base_cost_per_m2, scale_exp = 600, 0.85
    else:
        base_cost_per_m2, scale_exp = 350, 0.85
    
    n_shells = max(1, int(np.ceil(area_m2 / MAX_AREA)))
    area_per_shell = area_m2 / n_shells
    
    ref_area = 100
    Cp_ref = base_cost_per_m2 * ref_area
    Cp_per_shell = Cp_ref * (area_per_shell / ref_area) ** scale_exp
    Cp_total = Cp_per_shell * n_shells
    
    CEPCI_2022 = 816.0
    Cp_adj = Cp_total * (CEPCI_TARGET / CEPCI_2022)
    
    Fp = min(1.5, 1.0 + 0.02 * max(0, P_barg - 10))
    Cp_adj = Cp_adj * Fp
    
    B1, B2 = 1.63, 1.66
    F_BM = B1 + B2 * 1.0 * Fp
    C_BM = Cp_adj * F_BM
    
    return {
        'purchased': round(Cp_adj, 0),
        'installed': round(C_BM, 0),
        'n_shells': n_shells,
        'area_per_shell': round(area_per_shell, 1),
        'cost_per_m2': round(Cp_adj / area_m2, 0),
        'method': 'Industry data power law (Thunder Said 2023, Xylem 2024)'
    }


def calc_pem_electrolyzer(power_kW: float) -> Dict:
    """
    PEM electrolyzer cost using DOE H2NEW Consortium (2024) data.
    
    Reference: NREL/TP-5900-87625
    2024 value: $801/kW installed (after 10% learning from 2022 base of $890/kW)
    """
    power_kW = max(100, power_kW)
    cost_per_kW = 801
    Cp = power_kW * cost_per_kW
    F_BM = 1.12
    C_BM = Cp * F_BM
    
    return {
        'purchased': round(Cp, 0),
        'installed': round(C_BM, 0),
        'cost_per_kW': cost_per_kW,
        'method': 'DOE H2NEW (2024) NREL/TP-5900-87625'
    }


# DBD Plasma Reactor Reference Values (Table 3.5)
DBD_REFERENCE = {
    'n_tubes': 240,
    'power_kW': 114.7,
    'tube_cost_EUR': 15142.98,
    'power_supply_EUR': 173000,
    'tube_capacity_NLmin': 111,
    'SEI_kJ_m3': 15000,
    'cost_per_tube_effective': 63.10,
}


def calc_dbd_plasma_reactor(power_kW: float, flow_NLmin: float = None,
                             tube_capacity_NLmin: float = None) -> Dict:
    """
    DBD plasma reactor cost using THESIS methodology from Table 3.5.
    
    COST METHODOLOGY:
    =================
    
    Component 1: DBD Tube Bundle
    ----------------------------
    Thesis formula: C_tubes = €1/tube × n_tubes × 1000^0.6 = €63.10 × n_tubes
    For industrial scale (> 240 tubes): C = C_ref × (n_tubes / n_ref)^0.6
    
    Component 2: High-Voltage Power Supply
    --------------------------------------
    Pilot scale (< 1 MW): C = €1,500 × kW + €900 (linear)
    Industrial scale (> 1 MW): C = C_ref × (Power / P_ref)^0.7
    
    References:
    - Delikonstantis et al. (2017). Plasma-assisted nitrogen fixation
    - Martini et al. (2019). DBD reactor design and costing
    """
    power_kW = max(10, power_kW)
    
    if tube_capacity_NLmin is None:
        tube_capacity_NLmin = DBD_REFERENCE['tube_capacity_NLmin']
    
    if flow_NLmin is None or flow_NLmin <= 0:
        flow_NLmin = power_kW / 0.25
    
    n_tubes = max(1, int(np.ceil(flow_NLmin / tube_capacity_NLmin)))
    
    # Component 1: DBD Tube Bundle Cost
    n_ref = DBD_REFERENCE['n_tubes']
    C_ref_tubes = DBD_REFERENCE['tube_cost_EUR']
    
    if n_tubes <= n_ref:
        C_tubes_EUR = DBD_REFERENCE['cost_per_tube_effective'] * n_tubes
    else:
        C_tubes_EUR = C_ref_tubes * (n_tubes / n_ref) ** 0.6
    
    # Component 2: High-Voltage Power Supply Cost
    P_ref = DBD_REFERENCE['power_kW']
    C_ref_power = DBD_REFERENCE['power_supply_EUR']
    
    if power_kW <= 1000:
        C_power_EUR = 1500 * power_kW + 900
    else:
        C_power_EUR = C_ref_power * (power_kW / P_ref) ** 0.7
    
    # Total Cost
    C_total_EUR = C_tubes_EUR + C_power_EUR
    EUR_USD = 1.08
    Cp = C_total_EUR * EUR_USD
    
    F_BM = 1.8
    C_BM = Cp * F_BM
    
    return {
        'purchased': round(Cp, 0),
        'installed': round(C_BM, 0),
        'n_tubes': n_tubes,
        'flow_NLmin': round(flow_NLmin, 0),
        'tube_bundle_EUR': round(C_tubes_EUR, 0),
        'power_supply_EUR': round(C_power_EUR, 0),
        'total_EUR': round(C_total_EUR, 0),
        'cost_per_kW': round(Cp / power_kW, 0),
        'cost_per_tube': round(C_tubes_EUR / n_tubes, 2) if n_tubes > 0 else 0,
        'method': 'Thesis Table 3.5 (Delikonstantis 2017, Martini 2019)'
    }


def calc_haber_bosch_reactor(volume_m3: float, catalyst_tonnes: float,
                              P_barg: float = 200, production_kgh: float = None) -> Dict:
    """
    Haber-Bosch reactor cost for small-scale ammonia synthesis.
    
    References:
    - Ammonia Energy Association (2024): Small-scale ammonia economics
    - Smith & Torrente-Murciano (2020): Small-scale H-B process analysis
    """
    volume_m3 = max(0.01, volume_m3)
    catalyst_tonnes = max(0.01, catalyst_tonnes)
    
    if production_kgh is None:
        production_kgh = catalyst_tonnes * 1000 / 7.8 * 15000 / 22.4 * 0.15 * 17 / 1000
        production_kgh = max(10, production_kgh)
    
    production_tpd = production_kgh * 24 / 1000
    
    # Capacity-based scaling
    ref_capacity_tpd, ref_cost, scale_exp = 100, 3_000_000, 0.65
    C_scaled = ref_cost * (production_tpd / ref_capacity_tpd) ** scale_exp
    
    # Component-based estimate
    D_m = (4 * volume_m3 / (np.pi * 3.0)) ** (1/3)
    S, E = 103, 0.85
    t = (P_barg * 0.1 * D_m/2) / (S * E - 0.6 * P_barg * 0.1)
    t = max(0.025, t)
    
    L_m = volume_m3 / (np.pi/4 * D_m**2)
    rho = 7850
    D_outer = D_m + 2*t
    V_shell = np.pi/4 * (D_outer**2 - D_m**2) * L_m
    W_shell = V_shell * rho
    W_heads = 2 * np.pi/4 * D_outer**2 * (t * 1.5) * rho
    W_total = W_shell + W_heads
    
    cost_per_kg = 40
    C_vessel = max(150_000, W_total * cost_per_kg)
    C_catalyst = catalyst_tonnes * 15000
    C_internals = C_vessel * 0.75
    C_component = C_vessel + C_catalyst + C_internals
    
    Cp = max(600_000, max(C_scaled, C_component))
    
    F_BM = 2.5
    C_BM = Cp * F_BM
    
    return {
        'purchased': round(Cp, 0),
        'installed': round(C_BM, 0),
        'vessel_cost': round(C_vessel, 0),
        'catalyst_cost': round(C_catalyst, 0),
        'production_tpd': round(production_tpd, 2),
        'method': 'Small-scale H-B (Ammonia Energy 2024, scaled + floor)'
    }


def calc_combustion_chamber(duty_MW: float, volume_m3: float = None) -> Dict:
    """
    Combustion chamber cost for purge gas combustor.
    """
    duty_MW = max(0.5, duty_MW)
    
    if duty_MW < 5:
        burner_cost_per_MW = 4000
    elif duty_MW < 20:
        burner_cost_per_MW = 2500
    else:
        burner_cost_per_MW = 1500
    
    C_burner = max(10_000, duty_MW * burner_cost_per_MW)
    
    if volume_m3 is None:
        volume_m3 = duty_MW / 10
    
    C_chamber = 40_000 if volume_m3 < 2 else 30_000 * (volume_m3 ** 0.6)
    C_controls = 15_000 + duty_MW * 500
    C_stack = 10_000 + duty_MW * 300
    
    Cp = max(75_000, C_burner + C_chamber + C_controls + C_stack)
    
    F_BM = 2.5
    C_BM = Cp * F_BM
    
    return {
        'purchased': round(Cp, 0),
        'installed': round(C_BM, 0),
        'cost_per_MW': round(Cp / duty_MW, 0),
        'method': 'Component-based (burner + chamber + controls)'
    }


# =============================================================================
# SECTION 3: EQUIPMENT SIZING DATA (from process simulation)
# =============================================================================

# Equipment sizing parameters extracted from DWSIM simulation
EQUIPMENT_SIZING = {
    # Turbines (power in kW)
    'T-100': {'type': 'turbine', 'power_kW': 13472.97, 'desc': 'Steam turbine'},
    'T-101': {'type': 'turbine', 'power_kW': 11167.13, 'desc': 'NH3 vapor turbine'},
    'T-102': {'type': 'turbine', 'power_kW': 172.76, 'desc': 'Syngas expander'},
    
    # Compressors (power in kW)
    'K-100': {'type': 'compressor', 'power_kW': 6.41, 'desc': 'Low-pressure compressor'},
    'K-101': {'type': 'compressor', 'power_kW': 8.70, 'desc': 'Medium-pressure compressor'},
    'K-102': {'type': 'compressor', 'power_kW': 802.76, 'desc': 'High-pressure compressor'},
    
    # Pumps (power in kW, head in m)
    'P-100': {'type': 'pump', 'power_kW': 15.67, 'head_m': 50, 'desc': 'Water pump'},
    'P-101': {'type': 'pump', 'power_kW': 347.94, 'head_m': 200, 'desc': 'Ammonia pump'},
    
    # Heat Exchangers (area in m²)
    'E-100': {'type': 'heat_exchanger', 'area_m2': 150, 'P_barg': 10, 'hx_type': 'shell_tube', 'desc': 'Feed preheater'},
    'E-101': {'type': 'heat_exchanger', 'area_m2': 200, 'P_barg': 100, 'hx_type': 'shell_tube', 'desc': 'Reactor HX'},
    'E-102': {'type': 'heat_exchanger', 'area_m2': 100, 'P_barg': 10, 'hx_type': 'shell_tube', 'desc': 'Product cooler'},
    'E-103': {'type': 'heat_exchanger', 'area_m2': 500, 'P_barg': 5, 'hx_type': 'finned', 'desc': 'District heating HX'},
    'E-104': {'type': 'heat_exchanger', 'area_m2': 300, 'P_barg': 5, 'hx_type': 'shell_tube', 'desc': 'Condenser'},
    'E-105': {'type': 'heat_exchanger', 'area_m2': 80, 'P_barg': 100, 'hx_type': 'shell_tube', 'desc': 'Syngas HX'},
    'E-110': {'type': 'heat_exchanger', 'area_m2': 400, 'P_barg': 5, 'hx_type': 'finned', 'desc': 'WHRU'},
    'E-111': {'type': 'heat_exchanger', 'area_m2': 250, 'P_barg': 10, 'hx_type': 'shell_tube', 'desc': 'NH3 condenser'},
    
    # Vessels (volume in m³, diameter in m)
    'V-100': {'type': 'vessel', 'volume_m3': 5, 'diameter_m': 1.2, 'P_barg': 10, 'desc': 'Flash drum'},
    'V-101': {'type': 'vessel', 'volume_m3': 3, 'diameter_m': 1.0, 'P_barg': 100, 'desc': 'HP separator'},
    'V-102': {'type': 'vessel', 'volume_m3': 10, 'diameter_m': 1.5, 'P_barg': 5, 'desc': 'NH3 storage'},
    'V-103': {'type': 'vessel', 'volume_m3': 2, 'diameter_m': 0.8, 'P_barg': 20, 'desc': 'Buffer tank'},
    
    # Control Valves (Cv)
    'CV-100': {'type': 'control_valve', 'Cv': 50, 'desc': 'Feed control'},
    'CV-101': {'type': 'control_valve', 'Cv': 30, 'desc': 'Pressure control'},
    'CV-102': {'type': 'control_valve', 'Cv': 100, 'desc': 'Level control'},
    'CV-103': {'type': 'control_valve', 'Cv': 25, 'desc': 'Flow control'},
    
    # Special Equipment
    'DBD-100': {'type': 'dbd_plasma', 'power_kW': 12440, 'desc': 'DBD Plasma Reactor'},
    'PEM-100': {'type': 'pem_electrolyzer', 'power_kW': 5873, 'desc': 'PEM Electrolyzer'},
    'HB-100': {'type': 'haber_bosch', 'volume_m3': 0.5, 'catalyst_tonnes': 0.3, 'P_barg': 200, 'desc': 'H-B Reactor (not used in plasma route)'},
    'CC-100': {'type': 'combustion_chamber', 'duty_MW': 2.5, 'desc': 'Purge gas combustor'},
}


# =============================================================================
# SECTION 4: PROCESS PARAMETERS FROM STREAM DATA
# =============================================================================

# Operation
OPERATION_HOURS = 7300  # h/year
SYSTEM_LIFETIME = 20    # years
INTEREST_RATE = 0.14    # 14%

# INPUT MATERIALS (from stream data)
INPUTS = {
    'water_pem': {
        'stream': 13,
        'flow_kgmol_h': 50.0,
        'MW': 18.015,
        'flow_kg_h': 50.0 * 18.015,
        'annual_tonnes': 50.0 * 18.015 * 7300 / 1000,
    },
    'nitrogen': {
        'stream': 27,
        'flow_kgmol_h': 69.0,
        'MW': 28.014,
        'flow_kg_h': 69.0 * 28.014,
        'annual_tonnes': 69.0 * 28.014 * 7300 / 1000,
    },
    'air': {
        'stream': 55,
        'flow_kgmol_h': 62.39,
        'MW': 28.97,
        'flow_kg_h': 62.39 * 28.97,
        'annual_tonnes': 62.39 * 28.97 * 7300 / 1000,
        'cost_per_tonne': 0,
    },
}

# OUTPUT PRODUCTS
OUTPUTS = {
    'ammonia': {
        'flow_kg_h': 383.5,
        'annual_tonnes': 383.5 * 7300 / 1000,
    },
    'oxygen': {
        'stream': 25,
        'flow_kgmol_h': 25.0,
        'MW': 32.0,
        'flow_kg_h': 25.0 * 32.0,
        'annual_tonnes': 25.0 * 32.0 * 7300 / 1000,
    },
}

AMMONIA_TOTAL_KG_H = 383.5
AMMONIA_ANNUAL_TONNES = AMMONIA_TOTAL_KG_H * OPERATION_HOURS / 1000

# Energy streams
ENERGY = {
    'turbine_T100': 13472.97,
    'turbine_T101': 11167.13,
    'turbine_T102': 172.76,
    'pump_P100': -15.67,
    'pump_P101': -347.94,
    'compressor_K100': -6.41,
    'compressor_K101': -8.70,
    'compressor_K102': -802.76,
    'plasma_reactor': -12440,
}

DISTRICT_HEATING_KW = 50045

# Catalyst and Electrode Lifetimes
CATALYST_LIFETIMES = {
    'plasma_catalyst': 10,
    'pem_stack': 7,
}

CATALYST_COST_FRACTIONS = {
    'plasma_catalyst': 0.15,
    'pem_stack': 0.40,
}

# Market Prices
PRICES = {
    'demin_water': 3.0,
    'nitrogen_purchase': 80.0,
    'nitrogen_asu_power': 0.35,
    'ammonia': 870.0,
    'ammonia_grey': 480.0,
    'oxygen': 150.0,
    'electricity': 0.10,
    'heating': 12.0,
    'cooling_water': 0.354,
    'electricity_buy': 0.08,
}

# Technology Baseline
BASELINE_TECH = {
    'exergy_efficiency': 0.1539,
    'conversion': 0.10,
    'plasma_power_kW': 12440,
    'ammonia_kg_h': 383.5,
    'reactor_inlet_P_kPa': 10000,
    'energy_from_nh3_kW': 39320.4,
    'reactor_exergy_destruction_kW': 385,
}

EFFICIENCY_MULTIPLIERS = [1, 2, 3, 4, 5]


# =============================================================================
# SECTION 5: EQUIPMENT COST CALCULATION
# =============================================================================

def calculate_all_equipment_costs(exclude_haber_bosch: bool = True, 
                                   verbose: bool = True) -> Dict:
    """
    Calculate costs for all equipment using the defined sizing parameters
    and cost correlation functions.
    
    Parameters:
    -----------
    exclude_haber_bosch : bool
        If True, excludes the Haber-Bosch reactor (for plasma-only route)
    verbose : bool
        If True, prints detailed cost breakdown
    
    Returns:
    --------
    dict containing:
        - 'equipment_costs': List of individual equipment cost dictionaries
        - 'total_purchased': Total purchased equipment cost ($)
        - 'total_installed': Total installed equipment cost ($)
        - 'summary_by_type': Summary grouped by equipment type
    """
    equipment_costs = []
    
    if verbose:
        print("\n" + "=" * 80)
        print("EQUIPMENT COST CALCULATION (DYNAMIC USING COST FUNCTIONS)")
        print("=" * 80)
        print(f"Target Year: {TARGET_YEAR}, CEPCI: {CEPCI_TARGET}")
    
    for tag, params in EQUIPMENT_SIZING.items():
        eq_type = params['type']
        
        # Skip Haber-Bosch if requested (for plasma-only route)
        if exclude_haber_bosch and eq_type == 'haber_bosch':
            continue
        
        # Calculate cost based on equipment type
        if eq_type == 'turbine':
            result = calc_turbine(params['power_kW'])
            key_param = f"{params['power_kW']:.0f} kW"
            
        elif eq_type == 'compressor':
            result = calc_compressor(params['power_kW'])
            key_param = f"{params['power_kW']:.1f} kW"
            
        elif eq_type == 'pump':
            result = calc_pump(params['power_kW'], params.get('head_m', 100))
            key_param = f"{params['power_kW']:.1f} kW"
            
        elif eq_type == 'heat_exchanger':
            result = calc_heat_exchanger(
                params['area_m2'],
                params.get('hx_type', 'shell_tube'),
                params.get('P_barg', 10)
            )
            key_param = f"{params['area_m2']:.0f} m²"
            
        elif eq_type == 'vessel':
            result = calc_vessel(
                params['volume_m3'],
                params['diameter_m'],
                params.get('P_barg', 10)
            )
            key_param = f"{params['volume_m3']:.1f} m³"
            
        elif eq_type == 'control_valve':
            result = calc_control_valve(params['Cv'])
            key_param = f"Cv={params['Cv']:.0f}"
            
        elif eq_type == 'dbd_plasma':
            result = calc_dbd_plasma_reactor(params['power_kW'])
            key_param = f"{params['power_kW']:.0f} kW, {result['n_tubes']} tubes"
            
        elif eq_type == 'pem_electrolyzer':
            result = calc_pem_electrolyzer(params['power_kW'])
            key_param = f"{params['power_kW']:.0f} kW"
            
        elif eq_type == 'haber_bosch':
            result = calc_haber_bosch_reactor(
                params['volume_m3'],
                params['catalyst_tonnes'],
                params.get('P_barg', 200)
            )
            key_param = f"{params['volume_m3']:.2f} m³"
            
        elif eq_type == 'combustion_chamber':
            result = calc_combustion_chamber(params['duty_MW'])
            key_param = f"{params['duty_MW']:.1f} MW"
            
        else:
            continue
        
        equipment_costs.append({
            'Tag': tag,
            'Type': eq_type,
            'Description': params.get('desc', ''),
            'Key_Parameter': key_param,
            'Purchased': result['purchased'],
            'Installed': result['installed'],
            'Method': result['method']
        })
    
    # Calculate totals
    total_purchased = sum(eq['Purchased'] for eq in equipment_costs)
    total_installed = sum(eq['Installed'] for eq in equipment_costs)
    
    # Summary by type
    df = pd.DataFrame(equipment_costs)
    summary = df.groupby('Type').agg({
        'Purchased': 'sum',
        'Installed': 'sum',
        'Tag': 'count'
    }).rename(columns={'Tag': 'Count'})
    summary['Pct'] = (summary['Installed'] / total_installed * 100).round(1)
    summary = summary.sort_values('Installed', ascending=False)
    
    if verbose:
        print("\n[EQUIPMENT COST SUMMARY BY TYPE]")
        print("-" * 70)
        for eq_type, row in summary.iterrows():
            print(f"  {eq_type:25s}: ${row['Purchased']:>12,.0f} / ${row['Installed']:>12,.0f} "
                  f"({row['Count']:.0f} items, {row['Pct']:.1f}%)")
        print("-" * 70)
        print(f"  {'TOTAL':25s}: ${total_purchased:>12,.0f} / ${total_installed:>12,.0f}")
        print("=" * 80)
    
    return {
        'equipment_costs': equipment_costs,
        'total_purchased': total_purchased,
        'total_installed': total_installed,
        'summary_by_type': summary,
    }


def get_key_equipment_costs() -> Dict:
    """
    Get costs for key equipment items (plasma reactor, PEM electrolyzer)
    for use in operating cost calculations.
    """
    plasma = calc_dbd_plasma_reactor(EQUIPMENT_SIZING['DBD-100']['power_kW'])
    pem = calc_pem_electrolyzer(EQUIPMENT_SIZING['PEM-100']['power_kW'])
    
    return {
        'plasma_reactor_purchased': plasma['purchased'],
        'plasma_reactor_installed': plasma['installed'],
        'pem_electrolyzer_purchased': pem['purchased'],
        'pem_electrolyzer_installed': pem['installed'],
    }


# =============================================================================
# SECTION 6: CAPITAL INVESTMENT CALCULATION
# =============================================================================

def calculate_capital_investment(installed_cost: float, purchased_cost: float,
                                  include_asu: bool = True) -> dict:
    """
    Calculate Total Capital Investment from installed equipment cost.
    Optionally includes Air Separation Unit capital.
    """
    ISBL = installed_cost
    
    if include_asu:
        asu_capacity_tpd = 1933 * 24 / 1000
        asu_capital = asu_capacity_tpd * 250000
        print(f"  Including ASU capital: ${asu_capital/1e6:.1f}M for {asu_capacity_tpd:.0f} t/d N2")
        ISBL += asu_capital * 0.5
    
    # Additional direct costs
    site_development = 0.05 * ISBL
    land = 0.02 * ISBL
    buildings = 0.08 * ISBL
    utility_systems = 0.10 * ISBL
    total_direct = ISBL + site_development + land + buildings + utility_systems
    
    # Indirect costs
    engineering = 0.12 * ISBL
    construction_mgmt = 0.08 * ISBL
    contingencies = 0.18 * ISBL
    total_indirect = engineering + construction_mgmt + contingencies
    
    FCI = total_direct + total_indirect
    
    startup_costs = 0.08 * FCI
    TCI = (FCI + startup_costs) / 0.85
    working_capital = 0.15 * TCI
    
    return {
        'ISBL': ISBL,
        'PEC': purchased_cost,
        'site_development': site_development,
        'land': land,
        'buildings': buildings,
        'utility_systems': utility_systems,
        'total_direct': total_direct,
        'engineering': engineering,
        'construction_mgmt': construction_mgmt,
        'contingencies': contingencies,
        'total_indirect': total_indirect,
        'FCI': FCI,
        'startup_costs': startup_costs,
        'working_capital': working_capital,
        'TCI': TCI,
        'lang_factor': TCI / purchased_cost if purchased_cost else None,
    }


# =============================================================================
# SECTION 7: OPERATING COSTS
# =============================================================================

def calculate_operating_costs(FCI: float, nitrogen_source: str = 'purchase') -> dict:
    """
    Calculate annual operating costs including material inputs.
    Uses dynamically calculated equipment costs for catalyst replacement.
    """
    costs = {}
    
    # Raw Materials
    water_annual = INPUTS['water_pem']['annual_tonnes']
    costs['demin_water'] = water_annual * PRICES['demin_water']
    
    n2_annual = INPUTS['nitrogen']['annual_tonnes']
    if nitrogen_source == 'purchase':
        costs['nitrogen'] = n2_annual * PRICES['nitrogen_purchase']
    else:
        n2_kg_h = INPUTS['nitrogen']['flow_kg_h']
        asu_power_kW = n2_kg_h * PRICES['nitrogen_asu_power']
        costs['nitrogen'] = asu_power_kW * OPERATION_HOURS * PRICES['electricity_buy']
    
    # Utilities
    cooling_duty_kW = 44355 + 32000
    cooling_GJ_year = cooling_duty_kW * 3.6 * OPERATION_HOURS / 1e6
    costs['cooling_water'] = cooling_GJ_year * PRICES['cooling_water']
    
    # Fixed Operating Costs
    costs['maintenance'] = 0.025 * FCI
    costs['labor'] = 4 * 75000
    costs['insurance_taxes'] = 0.015 * FCI
    
    # Catalyst/Electrode Replacement (using dynamically calculated costs)
    key_costs = get_key_equipment_costs()
    
    plasma_catalyst_cost = key_costs['plasma_reactor_purchased'] * CATALYST_COST_FRACTIONS['plasma_catalyst']
    plasma_annual = plasma_catalyst_cost / CATALYST_LIFETIMES['plasma_catalyst']
    
    pem_stack_cost = key_costs['pem_electrolyzer_purchased'] * CATALYST_COST_FRACTIONS['pem_stack']
    pem_annual = pem_stack_cost / CATALYST_LIFETIMES['pem_stack']
    
    costs['catalyst_replacement'] = plasma_annual + pem_annual
    costs['total'] = sum(costs.values())
    
    return costs


# =============================================================================
# SECTION 8: REVENUE
# =============================================================================

def calculate_revenue() -> dict:
    """Calculate annual revenue from all products."""
    revenue = {}
    
    revenue['ammonia'] = AMMONIA_ANNUAL_TONNES * PRICES['ammonia']
    
    o2_annual = OUTPUTS['oxygen']['annual_tonnes']
    revenue['oxygen'] = o2_annual * PRICES['oxygen']
    
    gross_power = (ENERGY['turbine_T100'] + ENERGY['turbine_T101'] + ENERGY['turbine_T102'])
    parasitic = (abs(ENERGY['pump_P100']) + abs(ENERGY['pump_P101']) +
                 abs(ENERGY['compressor_K100']) + abs(ENERGY['compressor_K101']) +
                 abs(ENERGY['compressor_K102']) + abs(ENERGY['plasma_reactor']))
    
    net_power_kW = gross_power - parasitic
    revenue['net_power_kW'] = net_power_kW
    
    if net_power_kW > 0:
        revenue['electricity'] = net_power_kW * OPERATION_HOURS * PRICES['electricity']
    else:
        revenue['electricity'] = 0
    
    heating_GJ_year = DISTRICT_HEATING_KW * OPERATION_HOURS * 0.0036
    revenue['heating'] = heating_GJ_year * PRICES['heating']
    
    revenue['total'] = (revenue['ammonia'] + revenue['oxygen'] + 
                        revenue['electricity'] + revenue['heating'])
    
    return revenue


# =============================================================================
# SECTION 9: ECONOMIC CALCULATIONS
# =============================================================================

def calculate_CRF(i: float = INTEREST_RATE, n: int = SYSTEM_LIFETIME) -> float:
    """Capital Recovery Factor"""
    return (i * (1 + i)**n) / ((1 + i)**n - 1)


def calculate_NPV(net_income: float, TCI: float) -> float:
    """Net Present Value"""
    PVAF = ((1 + INTEREST_RATE)**SYSTEM_LIFETIME - 1) / \
           (INTEREST_RATE * (1 + INTEREST_RATE)**SYSTEM_LIFETIME)
    return net_income * PVAF - TCI


def calculate_payback(TCI: float, net_income: float) -> float:
    """Simple Payback Period"""
    return TCI / net_income if net_income > 0 else float('inf')


def calculate_LCOA(TCI: float, opex: float) -> float:
    """Levelized Cost of Ammonia"""
    CRF = calculate_CRF()
    return (TCI * CRF + opex) / AMMONIA_ANNUAL_TONNES


def calculate_IRR(TCI: float, net_income: float) -> float:
    """Internal Rate of Return"""
    def npv_at_rate(r):
        if r <= -1:
            return float('inf')
        pvaf = ((1 + r)**SYSTEM_LIFETIME - 1) / (r * (1 + r)**SYSTEM_LIFETIME) if r != 0 else SYSTEM_LIFETIME
        return net_income * pvaf - TCI
    
    low, high = -0.5, 2.0
    for _ in range(100):
        mid = (low + high) / 2
        npv = npv_at_rate(mid)
        if abs(npv) < 1:
            break
        if npv > 0:
            low = mid
        else:
            high = mid
    return mid


# =============================================================================
# SECTION 10: PLOTTING FUNCTIONS
# =============================================================================

def plot_cost_breakdown(opex: dict, revenue: dict, save_path: str):
    """Plot operating cost and revenue breakdown."""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
    
    cost_labels = ['N₂ Feed', 'Maintenance', 'Labor', 'Insurance', 
                   'Catalyst', 'Cooling', 'DM Water']
    cost_values = [opex['nitrogen'], opex['maintenance'], opex['labor'],
                   opex['insurance_taxes'], opex['catalyst_replacement'],
                   opex['cooling_water'], opex['demin_water']]
    
    colors1 = plt.cm.Reds(np.linspace(0.3, 0.8, len(cost_labels)))
    ax1.pie(cost_values, labels=cost_labels, autopct='%1.1f%%', colors=colors1)
    ax1.set_title(f'Annual OPEX: ${opex["total"]/1e6:.2f}M', fontweight='bold')
    
    rev_labels = ['Ammonia', 'Electricity', 'Oxygen', 'District Heat']
    rev_values = [revenue['ammonia'], revenue['electricity'], 
                  revenue['oxygen'], revenue['heating']]
    
    colors2 = plt.cm.Greens(np.linspace(0.3, 0.8, len(rev_labels)))
    ax2.pie(rev_values, labels=rev_labels, autopct='%1.1f%%', colors=colors2)
    ax2.set_title(f'Annual Revenue: ${revenue["total"]/1e6:.2f}M', fontweight='bold')
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()


def plot_economic_indicators(results: dict, save_path: str):
    """Plot key economic indicators."""
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    
    # NPV
    ax = axes[0, 0]
    npv_M = results['NPV'] / 1e6
    color = 'forestgreen' if npv_M > 0 else 'crimson'
    ax.bar(['NPV'], [npv_M], color=color, edgecolor='black', linewidth=2)
    ax.axhline(0, color='black', linewidth=1)
    ax.set_ylabel('Million $')
    ax.set_title(f'Net Present Value: ${npv_M:.1f}M', fontweight='bold')
    ax.grid(axis='y', alpha=0.3)
    
    # Payback
    ax = axes[0, 1]
    pp = min(results['PP'], SYSTEM_LIFETIME * 1.5)
    color = 'forestgreen' if results['PP'] < SYSTEM_LIFETIME else 'crimson'
    ax.bar(['Payback'], [pp], color=color, edgecolor='black', linewidth=2)
    ax.axhline(SYSTEM_LIFETIME, color='orange', linestyle='--', linewidth=2,
               label=f'Project Life ({SYSTEM_LIFETIME}y)')
    ax.set_ylabel('Years')
    pp_str = f'{results["PP"]:.1f}' if results['PP'] < 1000 else '∞'
    ax.set_title(f'Payback Period: {pp_str} years', fontweight='bold')
    ax.legend()
    ax.grid(axis='y', alpha=0.3)
    
    # LCOA
    ax = axes[1, 0]
    lcoa_display = min(results['LCOA'], PRICES['ammonia'] * 3)
    bars = ax.bar(['LCOA', 'Market'], [lcoa_display, PRICES['ammonia']],
                  color=['steelblue', 'forestgreen'], edgecolor='black', linewidth=2)
    ax.set_ylabel('$/tonne NH₃')
    ax.set_title(f'LCOA: ${results["LCOA"]:.0f} vs Market: ${PRICES["ammonia"]:.0f}', fontweight='bold')
    ax.grid(axis='y', alpha=0.3)
    
    # IRR
    ax = axes[1, 1]
    irr_pct = max(results['IRR'] * 100, -30)
    hurdle_pct = INTEREST_RATE * 100
    colors = ['forestgreen' if results['IRR'] > INTEREST_RATE else 'crimson', 'gray']
    ax.bar(['IRR', 'Hurdle'], [irr_pct, hurdle_pct], color=colors, edgecolor='black', linewidth=2)
    ax.axhline(0, color='black', linewidth=1)
    ax.set_ylabel('%')
    ax.set_title(f'IRR: {results["IRR"]*100:.1f}% vs Hurdle: {hurdle_pct:.0f}%', fontweight='bold')
    ax.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()


def plot_cashflow(TCI: float, net_income: float, save_path: str):
    """Plot cumulative cash flow."""
    years = np.arange(0, SYSTEM_LIFETIME + 1)
    
    cumulative = np.zeros(len(years))
    cumulative[0] = -TCI
    for i in range(1, len(years)):
        cumulative[i] = cumulative[i-1] + net_income
    
    cumulative_disc = np.zeros(len(years))
    cumulative_disc[0] = -TCI
    for i in range(1, len(years)):
        cumulative_disc[i] = cumulative_disc[i-1] + net_income / (1 + INTEREST_RATE)**i
    
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(years, cumulative/1e6, 'b-o', linewidth=2, markersize=5, label='Undiscounted')
    ax.plot(years, cumulative_disc/1e6, 'r-s', linewidth=2, markersize=5,
            label=f'Discounted (i={INTEREST_RATE*100:.0f}%)')
    ax.axhline(0, color='black', linewidth=1)
    ax.fill_between(years, 0, cumulative/1e6, where=cumulative > 0, alpha=0.2, color='green')
    ax.fill_between(years, 0, cumulative/1e6, where=cumulative < 0, alpha=0.2, color='red')
    
    ax.set_xlabel('Year', fontsize=12)
    ax.set_ylabel('Cumulative Cash Flow (Million $)', fontsize=12)
    ax.set_title('Project Cash Flow Analysis', fontsize=14, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()


def plot_material_balance(save_path: str):
    """Plot material balance diagram."""
    fig, ax = plt.subplots(figsize=(12, 8))
    
    inputs = {
        'N₂ (Stream 27)': INPUTS['nitrogen']['flow_kg_h'],
        'H₂O (Stream 13)': INPUTS['water_pem']['flow_kg_h'],
        'Air (Stream 55)': INPUTS['air']['flow_kg_h'],
    }
    
    outputs = {
        'NH₃ Product': AMMONIA_TOTAL_KG_H,
        'O₂ Byproduct': OUTPUTS['oxygen']['flow_kg_h'],
    }
    
    y_pos = np.arange(len(inputs))
    ax.barh(y_pos - 0.2, list(inputs.values()), height=0.35, 
            color='steelblue', label='Inputs', alpha=0.8)
    
    y_pos_out = np.arange(len(outputs))
    ax.barh(y_pos_out + len(inputs) + 0.3, list(outputs.values()), height=0.35,
            color='forestgreen', label='Products', alpha=0.8)
    
    all_labels = list(inputs.keys()) + [''] + list(outputs.keys())
    ax.set_yticks(list(range(len(inputs))) + [len(inputs)] + 
                  [i + len(inputs) + 1 for i in range(len(outputs))])
    ax.set_yticklabels(all_labels)
    
    ax.set_xlabel('Mass Flow (kg/h)', fontsize=12)
    ax.set_title('Material Balance (kg/h)', fontsize=14, fontweight='bold')
    ax.legend()
    ax.grid(axis='x', alpha=0.3)
    
    for i, (k, v) in enumerate(inputs.items()):
        ax.text(v + 50, i - 0.2, f'{v:.0f} kg/h', va='center', fontweight='bold')
    for i, (k, v) in enumerate(outputs.items()):
        ax.text(v + 50, i + len(inputs) + 0.8, f'{v:.0f} kg/h', va='center', fontweight='bold')
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()


def plot_equipment_cost_breakdown(equipment_results: Dict, save_path: str):
    """Plot equipment cost breakdown by type."""
    summary = equipment_results['summary_by_type']
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
    
    # Purchased costs
    colors = plt.cm.viridis(np.linspace(0.2, 0.8, len(summary)))
    ax1.pie(summary['Purchased'], labels=summary.index, autopct='%1.1f%%', colors=colors)
    ax1.set_title(f'Purchased Equipment Cost: ${equipment_results["total_purchased"]/1e6:.1f}M', 
                  fontweight='bold')
    
    # Installed costs
    ax2.pie(summary['Installed'], labels=summary.index, autopct='%1.1f%%', colors=colors)
    ax2.set_title(f'Installed Equipment Cost: ${equipment_results["total_installed"]/1e6:.1f}M', 
                  fontweight='bold')
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()


# =============================================================================
# SECTION 11: MAIN ANALYSIS
# =============================================================================

def run_complete_analysis(nitrogen_source: str = 'purchase',
                          include_asu: bool = False,
                          verbose: bool = True) -> dict:
    """
    Run complete economic analysis with dynamically calculated equipment costs.
    
    Parameters:
    -----------
    nitrogen_source: 'purchase' (buy LN2) or 'asu' (on-site production)
    include_asu: Include ASU capital cost if True
    verbose: Print detailed output
    """
    
    print("=" * 80)
    print("COMPLETE ECONOMIC ANALYSIS - PLASMA AMMONIA WITH ALL MATERIAL INPUTS")
    print("=" * 80)
    print("\n*** EQUIPMENT COSTS CALCULATED DYNAMICALLY USING VALIDATED CORRELATIONS ***\n")
    
    # Step 1: Calculate all equipment costs dynamically
    equipment_results = calculate_all_equipment_costs(
        exclude_haber_bosch=True,  # Plasma-only route
        verbose=verbose
    )
    
    PEC = equipment_results['total_purchased']
    ISBL = equipment_results['total_installed']
    
    # Material balance summary
    if verbose:
        print("\n[0] MATERIAL BALANCE (from stream data)")
        print("-" * 60)
        print("  INPUTS:")
        print(f"    Demineralized Water (S13): {INPUTS['water_pem']['flow_kg_h']:.1f} kg/h → {INPUTS['water_pem']['annual_tonnes']:.0f} t/yr")
        print(f"    Nitrogen Feed (S27):       {INPUTS['nitrogen']['flow_kg_h']:.1f} kg/h → {INPUTS['nitrogen']['annual_tonnes']:.0f} t/yr")
        print(f"    Air for Combustion (S55):  {INPUTS['air']['flow_kg_h']:.1f} kg/h (FREE)")
        print("  OUTPUTS:")
        print(f"    Ammonia Product (S46): {AMMONIA_TOTAL_KG_H:.1f} kg/h → {AMMONIA_ANNUAL_TONNES:.0f} t/yr")
        print(f"    Oxygen Byproduct (S25):    {OUTPUTS['oxygen']['flow_kg_h']:.1f} kg/h → {OUTPUTS['oxygen']['annual_tonnes']:.0f} t/yr")
    
    # Step 2: Capital investment
    if verbose:
        print(f"\n[1] CAPITAL INVESTMENT")
        print("-" * 60)
        print(f"  Nitrogen source: {nitrogen_source.upper()}")
    
    capital = calculate_capital_investment(ISBL, PEC, include_asu=include_asu)
    
    if verbose:
        print(f"\n  Purchased Equipment (PEC):   ${PEC:>15,.0f}")
        print(f"  Installed Equipment (ISBL):  ${capital['ISBL']:>15,.0f}")
        print(f"  Total Direct Costs:          ${capital['total_direct']:>15,.0f}")
        print(f"  Total Indirect Costs:        ${capital['total_indirect']:>15,.0f}")
        print(f"  Fixed Capital Investment:    ${capital['FCI']:>15,.0f}")
        print(f"  Startup + Working Capital:   ${capital['startup_costs'] + capital['working_capital']:>15,.0f}")
        print(f"  TOTAL CAPITAL INVESTMENT:    ${capital['TCI']:>15,.0f}")
        print(f"  Lang Factor (TCI/PEC):       {capital['lang_factor']:>15.2f}")
    
    # Step 3: Operating costs
    opex = calculate_operating_costs(capital['FCI'], nitrogen_source)
    
    if verbose:
        print(f"\n[2] ANNUAL OPERATING COSTS")
        print("-" * 60)
        print(f"  Raw Materials:")
        print(f"    Demineralized Water:       ${opex['demin_water']:>15,.0f}")
        print(f"    Nitrogen ({nitrogen_source}):       ${opex['nitrogen']:>15,.0f}")
        print(f"  Utilities:")
        print(f"    Cooling Water:             ${opex['cooling_water']:>15,.0f}")
        print(f"  Fixed Costs:")
        print(f"    Maintenance (2.5% FCI):    ${opex['maintenance']:>15,.0f}")
        print(f"    Labor (4 operators):       ${opex['labor']:>15,.0f}")
        print(f"    Insurance & Taxes:         ${opex['insurance_taxes']:>15,.0f}")
        print(f"    Catalyst/Electrodes:       ${opex['catalyst_replacement']:>15,.0f}")
        print(f"  TOTAL ANNUAL OPEX:           ${opex['total']:>15,.0f}")
    
    # Step 4: Revenue
    revenue = calculate_revenue()
    
    if verbose:
        print(f"\n[3] ANNUAL REVENUE")
        print("-" * 60)
        print(f"  Ammonia ({AMMONIA_ANNUAL_TONNES:.0f} t/yr × ${PRICES['ammonia']}/t):")
        print(f"                               ${revenue['ammonia']:>15,.0f}")
        print(f"  Oxygen ({OUTPUTS['oxygen']['annual_tonnes']:.0f} t/yr × ${PRICES['oxygen']}/t):")
        print(f"                               ${revenue['oxygen']:>15,.0f}")
        print(f"  Electricity ({revenue['net_power_kW']:.0f} kW net):")
        print(f"                               ${revenue['electricity']:>15,.0f}")
        print(f"  District Heating:")
        print(f"                               ${revenue['heating']:>15,.0f}")
        print(f"  TOTAL ANNUAL REVENUE:        ${revenue['total']:>15,.0f}")
    
    # Step 5: Economic indicators
    net_income = revenue['total'] - opex['total']
    CRF = calculate_CRF()
    PP = calculate_payback(capital['TCI'], net_income)
    NPV = calculate_NPV(net_income, capital['TCI'])
    IRR = calculate_IRR(capital['TCI'], net_income)
    LCOA = calculate_LCOA(capital['TCI'], opex['total'])
    
    if verbose:
        print(f"\n[4] ECONOMIC INDICATORS")
        print("-" * 60)
        print(f"  Capital Recovery Factor:     {CRF:>15.4f}")
        print(f"  Net Annual Income:           ${net_income:>15,.0f}")
        pp_str = f'{PP:.1f}' if PP < 1000 else '∞'
        print(f"  Payback Period:              {pp_str:>15} years")
        print(f"  Net Present Value:           ${NPV:>15,.0f}")
        print(f"  Internal Rate of Return:     {IRR*100:>15.1f}%")
        print(f"  Levelized Cost of Ammonia:   ${LCOA:>15,.0f}/tonne")
    
        print(f"\n[5] PROJECT VIABILITY")
        print("-" * 60)
        v_npv = NPV > 0
        v_irr = IRR > INTEREST_RATE
        v_lcoa = LCOA < PRICES['ammonia']
        v_pp = PP < SYSTEM_LIFETIME
        
        print(f"  NPV > 0:                     {'✓ YES' if v_npv else '✗ NO':>15}")
        print(f"  IRR > {INTEREST_RATE*100:.0f}%:                    {'✓ YES' if v_irr else '✗ NO':>15}")
        print(f"  LCOA < ${PRICES['ammonia']}/t:               {'✓ YES' if v_lcoa else '✗ NO':>15}")
        print(f"  PP < {SYSTEM_LIFETIME} years:                {'✓ YES' if v_pp else '✗ NO':>15}")
        print(f"\n  PROJECT VIABILITY:           {'VIABLE' if (v_npv and v_irr) else 'NOT VIABLE':>15}")
        print("=" * 80)
    
    results = {
        'equipment_results': equipment_results,
        'capital': capital,
        'TCI': capital['TCI'],
        'FCI': capital['FCI'],
        'PEC': PEC,
        'ISBL': capital['ISBL'],
        'opex': opex,
        'opex_total': opex['total'],
        'revenue': revenue,
        'revenue_total': revenue['total'],
        'net_income': net_income,
        'CRF': CRF,
        'PP': PP,
        'NPV': NPV,
        'IRR': IRR,
        'LCOA': LCOA,
        'ammonia_tpy': AMMONIA_ANNUAL_TONNES,
    }
    
    return results


# =============================================================================
# SECTION 12: TECHNOLOGY SENSITIVITY ANALYSIS
# =============================================================================

def calculate_technology_sensitivity(eff_multiplier: float, 
                                      baseline_results: dict) -> dict:
    """
    Calculate economics with improved plasma technology.
    
    With X× efficiency improvement:
    - NH3 production increases X×
    - N2 and H2O consumption increase X×
    - O2 byproduct increases X×
    - Equipment costs recalculated based on new sizing
    """
    
    new_efficiency = BASELINE_TECH['exergy_efficiency'] * eff_multiplier
    new_conversion = min(0.95, BASELINE_TECH['conversion'] * eff_multiplier)
    
    print(f"\n{'='*70}")
    print(f"TECHNOLOGY SCENARIO: {eff_multiplier}× Efficiency/Conversion")
    print(f"{'='*70}")
    print(f"  Baseline exergy efficiency: {BASELINE_TECH['exergy_efficiency']*100:.1f}%")
    print(f"  New exergy efficiency:      {new_efficiency*100:.1f}%")
    print(f"  Baseline conversion:        {BASELINE_TECH['conversion']*100:.0f}%")
    print(f"  New conversion:             {new_conversion*100:.0f}%")
    print(f"  Plasma power (unchanged):   {BASELINE_TECH['plasma_power_kW']:,.0f} kW")
    
    # Production Changes
    new_ammonia_kg_h = BASELINE_TECH['ammonia_kg_h'] * eff_multiplier
    new_ammonia_tpy = new_ammonia_kg_h * OPERATION_HOURS / 1000
    
    n2_multiplier = eff_multiplier
    h2o_multiplier = eff_multiplier
    o2_multiplier = eff_multiplier
    
    new_n2_tpy = INPUTS['nitrogen']['annual_tonnes'] * n2_multiplier
    new_h2o_tpy = INPUTS['water_pem']['annual_tonnes'] * h2o_multiplier
    new_o2_tpy = OUTPUTS['oxygen']['annual_tonnes'] * o2_multiplier
    
    print(f"\n  PRODUCTION:")
    print(f"    NH3: {BASELINE_TECH['ammonia_kg_h']:.1f} → {new_ammonia_kg_h:.1f} kg/h")
    print(f"         {AMMONIA_ANNUAL_TONNES:.0f} → {new_ammonia_tpy:.0f} t/yr")
    
    # Capital Cost Changes (using cost functions)
    # PEM Electrolyzer - must be larger for more H2 production
    pem_base = calc_pem_electrolyzer(EQUIPMENT_SIZING['PEM-100']['power_kW'])
    pem_new_power = EQUIPMENT_SIZING['PEM-100']['power_kW'] * h2o_multiplier
    pem_new = calc_pem_electrolyzer(pem_new_power)
    
    # Plasma reactor - SAME SIZE, but learning curve cost reduction
    plasma_base = calc_dbd_plasma_reactor(EQUIPMENT_SIZING['DBD-100']['power_kW'])
    learning_factor = 0.85 ** np.log2(max(1, eff_multiplier))
    plasma_new_purchased = plasma_base['purchased'] * learning_factor
    plasma_new_installed = plasma_base['installed'] * learning_factor
    
    # Compressors - lower recycle with higher conversion
    recycle_base = (1 - BASELINE_TECH['conversion']) / BASELINE_TECH['conversion']
    recycle_new = (1 - new_conversion) / new_conversion
    recycle_factor = recycle_new / recycle_base
    
    # Calculate baseline compressor costs
    comp_base_total = 0
    for tag in ['K-100', 'K-101', 'K-102']:
        comp = calc_compressor(EQUIPMENT_SIZING[tag]['power_kW'])
        comp_base_total += comp['purchased']
    
    comp_new_total = comp_base_total * (recycle_factor ** 0.6)
    
    # Other equipment unchanged
    other_equipment_cost = (baseline_results['PEC'] - 
                           pem_base['purchased'] - 
                           plasma_base['purchased'] - 
                           comp_base_total)
    
    new_PEC = (pem_new['purchased'] + 
               plasma_new_purchased + 
               comp_new_total + 
               other_equipment_cost)
    
    print(f"\n  CAPITAL CHANGES:")
    print(f"    PEM Electrolyzer: ${pem_base['purchased']/1e6:.2f}M → ${pem_new['purchased']/1e6:.2f}M ({h2o_multiplier:.0f}× capacity)")
    print(f"    Plasma Reactor:   ${plasma_base['purchased']/1e6:.2f}M → ${plasma_new_purchased/1e6:.2f}M (learning: {learning_factor:.2f}×)")
    print(f"    Compressors:      ${comp_base_total/1e3:.0f}k → ${comp_new_total/1e3:.0f}k (recycle: {recycle_factor:.2f}×)")
    print(f"    TOTAL PEC:        ${baseline_results['PEC']/1e6:.2f}M → ${new_PEC/1e6:.2f}M")
    
    # Calculate installed cost and TCI
    avg_BM = baseline_results['ISBL'] / baseline_results['PEC']
    new_ISBL = new_PEC * avg_BM
    
    capital = calculate_capital_investment(new_ISBL, new_PEC, include_asu=False)
    TCI = capital['TCI']
    FCI = capital['FCI']
    
    print(f"    TCI:              ${baseline_results['TCI']/1e6:.1f}M → ${TCI/1e6:.1f}M")
    
    # Operating Costs
    opex = {}
    opex['nitrogen'] = new_n2_tpy * PRICES['nitrogen_purchase']
    opex['demin_water'] = new_h2o_tpy * PRICES['demin_water']
    opex['maintenance'] = 0.025 * FCI
    opex['labor'] = 4 * 75000
    opex['insurance_taxes'] = 0.015 * FCI
    
    plasma_catalyst_cost = plasma_new_purchased * CATALYST_COST_FRACTIONS['plasma_catalyst']
    plasma_annual = plasma_catalyst_cost / CATALYST_LIFETIMES['plasma_catalyst']
    pem_stack_cost = pem_new['purchased'] * CATALYST_COST_FRACTIONS['pem_stack']
    pem_annual = pem_stack_cost / CATALYST_LIFETIMES['pem_stack']
    opex['catalyst_replacement'] = plasma_annual + pem_annual
    
    opex['cooling_water'] = baseline_results['opex']['cooling_water'] * eff_multiplier
    opex['total'] = sum(opex.values())
    
    # Revenue
    revenue = {}
    revenue['ammonia'] = new_ammonia_tpy * PRICES['ammonia']
    revenue['oxygen'] = new_o2_tpy * PRICES['oxygen']
    
    turbine_T100 = ENERGY['turbine_T100']
    turbine_T101 = ENERGY['turbine_T101'] * eff_multiplier
    turbine_T102 = ENERGY['turbine_T102'] * eff_multiplier
    
    pump_power = (abs(ENERGY['pump_P100']) + abs(ENERGY['pump_P101'])) * eff_multiplier
    comp_power = (abs(ENERGY['compressor_K100']) + abs(ENERGY['compressor_K101']) + 
                  abs(ENERGY['compressor_K102'])) * recycle_factor
    plasma_power = abs(ENERGY['plasma_reactor'])
    pem_power = pem_new_power
    
    gross_power = turbine_T100 + turbine_T101 + turbine_T102
    parasitic = pump_power + comp_power + plasma_power + pem_power
    net_power = gross_power - parasitic
    
    revenue['net_power_kW'] = net_power
    if net_power > 0:
        revenue['electricity'] = net_power * OPERATION_HOURS * PRICES['electricity']
    else:
        revenue['electricity'] = 0
        opex['electricity_purchase'] = abs(net_power) * OPERATION_HOURS * PRICES['electricity_buy']
        opex['total'] += opex.get('electricity_purchase', 0)
    
    revenue['heating'] = baseline_results['revenue']['heating'] * eff_multiplier
    revenue['total'] = (revenue['ammonia'] + revenue['oxygen'] + 
                        revenue['electricity'] + revenue['heating'])
    
    print(f"\n  POWER BALANCE:")
    print(f"    Turbines: {gross_power:.0f} kW")
    print(f"    Consumers: {parasitic:.0f} kW")
    print(f"    NET POWER: {net_power:.0f} kW ({'export' if net_power > 0 else 'import'})")
    
    # Economic Indicators
    net_income = revenue['total'] - opex['total']
    CRF = calculate_CRF()
    NPV = calculate_NPV(net_income, TCI)
    IRR = calculate_IRR(TCI, net_income)
    PP = calculate_payback(TCI, net_income)
    LCOA = (TCI * CRF + opex['total']) / new_ammonia_tpy
    
    print(f"\n  ECONOMIC INDICATORS:")
    print(f"    Net Income:  ${net_income/1e6:.2f}M/yr")
    print(f"    NPV:         ${NPV/1e6:.1f}M")
    print(f"    IRR:         {IRR*100:.1f}%")
    pp_str = f'{PP:.1f}' if PP < 100 else '∞'
    print(f"    Payback:     {pp_str} years")
    print(f"    LCOA:        ${LCOA:.0f}/t (Market: ${PRICES['ammonia']}/t)")
    
    viable = NPV > 0 and IRR > INTEREST_RATE and LCOA < PRICES['ammonia']
    print(f"\n  VIABILITY: {'✓ VIABLE' if viable else '✗ NOT VIABLE'}")
    
    return {
        'efficiency_factor': eff_multiplier,
        'new_efficiency': new_efficiency,
        'new_conversion': new_conversion,
        'ammonia_kg_h': new_ammonia_kg_h,
        'ammonia_tpy': new_ammonia_tpy,
        'n2_tpy': new_n2_tpy,
        'o2_tpy': new_o2_tpy,
        'PEC': new_PEC,
        'TCI': TCI,
        'FCI': FCI,
        'opex': opex,
        'opex_total': opex['total'],
        'revenue': revenue,
        'revenue_total': revenue['total'],
        'net_income': net_income,
        'net_power_kW': net_power,
        'NPV': NPV,
        'IRR': IRR,
        'PP': PP,
        'LCOA': LCOA,
        'viable': viable,
    }


def run_technology_sensitivity(baseline_results: dict) -> list:
    """Run technology improvement sensitivity for 1× to 5× efficiency."""
    print("\n" + "="*80)
    print("TECHNOLOGY IMPROVEMENT SENSITIVITY ANALYSIS")
    print("="*80)
    print("\nAssumption: Same plasma reactor (12,440 kW), improved efficiency")
    print("            Higher efficiency → more NH3 per kWh input")
    
    results = []
    for mult in EFFICIENCY_MULTIPLIERS:
        result = calculate_technology_sensitivity(mult, baseline_results)
        results.append(result)
    
    # Summary table
    print("\n" + "="*80)
    print("SENSITIVITY ANALYSIS SUMMARY")
    print("="*80)
    
    print(f"\n{'Eff.':<6} {'η (%)':<8} {'Conv.':<8} {'NH3':<10} {'TCI':<10} {'OPEX':<10} {'Revenue':<10} {'LCOA':<10} {'NPV':<12} {'IRR':<8} {'PP':<8} {'Status':<10}")
    print("-" * 120)
    
    for r in results:
        pp_str = f"{r['PP']:.1f}" if r['PP'] < 100 else "∞"
        status = "✓ VIABLE" if r['viable'] else "✗ NO"
        print(f"{r['efficiency_factor']:<6.0f}× {r['new_efficiency']*100:<8.1f} {r['new_conversion']*100:<8.0f} "
              f"{r['ammonia_tpy']:<10,.0f} {r['TCI']/1e6:<10.1f} {r['opex_total']/1e6:<10.2f} "
              f"{r['revenue_total']/1e6:<10.2f} {r['LCOA']:<10,.0f} {r['NPV']/1e6:<12.1f} "
              f"{r['IRR']*100:<8.1f} {pp_str:<8} {status:<10}")
    
    return results


def plot_technology_sensitivity(results: list, save_path: str):
    """Plot technology sensitivity results."""
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    
    eff_factors = [r['efficiency_factor'] for r in results]
    
    # LCOA vs Efficiency
    ax = axes[0, 0]
    lcoa_vals = [r['LCOA'] for r in results]
    colors = ['forestgreen' if l < PRICES['ammonia'] else 'crimson' for l in lcoa_vals]
    bars = ax.bar(eff_factors, lcoa_vals, color=colors, edgecolor='black', linewidth=1.5)
    ax.axhline(PRICES['ammonia'], color='orange', linestyle='--', linewidth=2,
               label=f"Green NH₃ Price (${PRICES['ammonia']}/t)")
    ax.set_xlabel('Efficiency Improvement Factor', fontsize=11)
    ax.set_ylabel('LCOA ($/tonne)', fontsize=11)
    ax.set_title('Levelized Cost of Ammonia', fontsize=12, fontweight='bold')
    ax.legend(loc='upper right')
    ax.set_xticks(eff_factors)
    ax.set_xticklabels([f'{x}×' for x in eff_factors])
    ax.grid(axis='y', alpha=0.3)
    
    # NPV vs Efficiency
    ax = axes[0, 1]
    npv_vals = [r['NPV']/1e6 for r in results]
    colors = ['forestgreen' if n > 0 else 'crimson' for n in npv_vals]
    ax.bar(eff_factors, npv_vals, color=colors, edgecolor='black', linewidth=1.5)
    ax.axhline(0, color='black', linewidth=1)
    ax.set_xlabel('Efficiency Improvement Factor', fontsize=11)
    ax.set_ylabel('NPV (Million $)', fontsize=11)
    ax.set_title('Net Present Value', fontsize=12, fontweight='bold')
    ax.set_xticks(eff_factors)
    ax.set_xticklabels([f'{x}×' for x in eff_factors])
    ax.grid(axis='y', alpha=0.3)
    
    # IRR vs Efficiency
    ax = axes[0, 2]
    irr_vals = [r['IRR']*100 for r in results]
    colors = ['forestgreen' if i > INTEREST_RATE*100 else 'crimson' for i in irr_vals]
    ax.bar(eff_factors, irr_vals, color=colors, edgecolor='black', linewidth=1.5)
    ax.axhline(INTEREST_RATE*100, color='orange', linestyle='--', linewidth=2,
               label=f"Hurdle Rate ({INTEREST_RATE*100:.0f}%)")
    ax.axhline(0, color='black', linewidth=1)
    ax.set_xlabel('Efficiency Improvement Factor', fontsize=11)
    ax.set_ylabel('IRR (%)', fontsize=11)
    ax.set_title('Internal Rate of Return', fontsize=12, fontweight='bold')
    ax.legend(loc='lower right')
    ax.set_xticks(eff_factors)
    ax.set_xticklabels([f'{x}×' for x in eff_factors])
    ax.grid(axis='y', alpha=0.3)
    
    # Production
    ax = axes[1, 0]
    nh3_vals = [r['ammonia_tpy'] for r in results]
    ax.bar(eff_factors, nh3_vals, color='steelblue', edgecolor='black', linewidth=1.5)
    ax.set_xlabel('Efficiency Improvement Factor', fontsize=11)
    ax.set_ylabel('NH₃ Production (t/yr)', fontsize=11)
    ax.set_title('Ammonia Production', fontsize=12, fontweight='bold')
    ax.set_xticks(eff_factors)
    ax.set_xticklabels([f'{x}×' for x in eff_factors])
    ax.grid(axis='y', alpha=0.3)
    
    # Payback Period
    ax = axes[1, 1]
    pp_vals = [min(r['PP'], 100) for r in results]
    colors = ['forestgreen' if r['PP'] < SYSTEM_LIFETIME else 'crimson' for r in results]
    ax.bar(eff_factors, pp_vals, color=colors, edgecolor='black', linewidth=1.5)
    ax.axhline(SYSTEM_LIFETIME, color='orange', linestyle='--', linewidth=2,
               label=f"Project Life ({SYSTEM_LIFETIME} yr)")
    ax.set_xlabel('Efficiency Improvement Factor', fontsize=11)
    ax.set_ylabel('Payback Period (years)', fontsize=11)
    ax.set_title('Simple Payback Period', fontsize=12, fontweight='bold')
    ax.legend(loc='upper right')
    ax.set_xticks(eff_factors)
    ax.set_xticklabels([f'{x}×' for x in eff_factors])
    ax.set_ylim([0, 80])
    ax.grid(axis='y', alpha=0.3)
    
    # Net Power Balance
    ax = axes[1, 2]
    power_vals = [r['net_power_kW'] for r in results]
    colors = ['forestgreen' if p > 0 else 'crimson' for p in power_vals]
    ax.bar(eff_factors, power_vals, color=colors, edgecolor='black', linewidth=1.5)
    ax.axhline(0, color='black', linewidth=1)
    ax.set_xlabel('Efficiency Improvement Factor', fontsize=11)
    ax.set_ylabel('Net Power (kW)', fontsize=11)
    ax.set_title('Net Power Balance (+ = export)', fontsize=12, fontweight='bold')
    ax.set_xticks(eff_factors)
    ax.set_xticklabels([f'{x}×' for x in eff_factors])
    ax.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"  Saved: {save_path}")


# =============================================================================
# SECTION 13: EXCEL OUTPUT
# =============================================================================

def save_to_excel(results: dict, tech_results: list = None, filename: str = 'Economic_Analysis_Integrated.xlsx'):
    """Save complete results to Excel with multiple sheets."""
    
    with pd.ExcelWriter(filename, engine='openpyxl') as writer:
        
        # Summary sheet
        summary = pd.DataFrame({
            'Parameter': [
                'Ammonia Production', 'Oxygen Byproduct', 'Net Power',
                '', 'PEC', 'ISBL', 'FCI', 'TCI', 'Lang Factor',
                '', 'Total OPEX', '  - Nitrogen', '  - Maintenance', '  - Labor',
                '  - Catalyst/Electrodes', '  - Other',
                '', 'Total Revenue', '  - Ammonia', '  - Oxygen', '  - Electricity', '  - Heating',
                '', 'Net Annual Income', 'Payback Period', 'NPV', 'IRR', 'LCOA'
            ],
            'Value': [
                AMMONIA_ANNUAL_TONNES, OUTPUTS['oxygen']['annual_tonnes'], 
                results['revenue']['net_power_kW'],
                '', results['PEC'], results['ISBL'], results['FCI'], results['TCI'],
                results['capital']['lang_factor'],
                '', results['opex_total'], results['opex']['nitrogen'],
                results['opex']['maintenance'], results['opex']['labor'],
                results['opex']['catalyst_replacement'],
                results['opex']['demin_water'] + results['opex']['cooling_water'] + results['opex']['insurance_taxes'],
                '', results['revenue_total'], results['revenue']['ammonia'],
                results['revenue']['oxygen'], results['revenue']['electricity'],
                results['revenue']['heating'],
                '', results['net_income'], results['PP'], results['NPV'],
                results['IRR'] * 100, results['LCOA']
            ],
            'Unit': [
                't/yr', 't/yr', 'kW',
                '', '$', '$', '$', '$', '-',
                '', '$/yr', '$/yr', '$/yr', '$/yr', '$/yr', '$/yr',
                '', '$/yr', '$/yr', '$/yr', '$/yr', '$/yr',
                '', '$/yr', 'years', '$', '%', '$/t'
            ]
        })
        summary.to_excel(writer, sheet_name='Summary', index=False)
        
        # Equipment Costs sheet
        eq_df = pd.DataFrame(results['equipment_results']['equipment_costs'])
        eq_df.to_excel(writer, sheet_name='Equipment Costs', index=False)
        
        # Equipment Summary sheet
        results['equipment_results']['summary_by_type'].to_excel(writer, sheet_name='Equipment Summary')
        
        # Material Balance sheet
        materials = pd.DataFrame({
            'Stream': ['Input: Water (S13)', 'Input: N2 (S27)', 'Input: Air (S55)',
                       'Output: NH3', 'Output: O2 (S25)'],
            'Flow (kg/h)': [INPUTS['water_pem']['flow_kg_h'], INPUTS['nitrogen']['flow_kg_h'],
                            INPUTS['air']['flow_kg_h'], AMMONIA_TOTAL_KG_H,
                            OUTPUTS['oxygen']['flow_kg_h']],
            'Annual (t/yr)': [INPUTS['water_pem']['annual_tonnes'], INPUTS['nitrogen']['annual_tonnes'],
                              INPUTS['air']['annual_tonnes'], AMMONIA_ANNUAL_TONNES,
                              OUTPUTS['oxygen']['annual_tonnes']],
            'Unit Cost ($/t)': [PRICES['demin_water'], PRICES['nitrogen_purchase'], 0,
                                PRICES['ammonia'], PRICES['oxygen']],
            'Annual Cost/Revenue ($)': [
                INPUTS['water_pem']['annual_tonnes'] * PRICES['demin_water'],
                INPUTS['nitrogen']['annual_tonnes'] * PRICES['nitrogen_purchase'],
                0, 
                AMMONIA_ANNUAL_TONNES * PRICES['ammonia'],
                OUTPUTS['oxygen']['annual_tonnes'] * PRICES['oxygen']
            ]
        })
        materials.to_excel(writer, sheet_name='Material Balance', index=False)
        
        # Technology Sensitivity sheet (if provided)
        if tech_results:
            tech_df = pd.DataFrame([{
                'Efficiency Factor': r['efficiency_factor'],
                'Exergy Efficiency (%)': r['new_efficiency'] * 100,
                'Conversion (%)': r['new_conversion'] * 100,
                'NH3 Production (t/yr)': r['ammonia_tpy'],
                'N2 Consumption (t/yr)': r['n2_tpy'],
                'O2 Byproduct (t/yr)': r['o2_tpy'],
                'Net Power (kW)': r['net_power_kW'],
                'PEC ($M)': r['PEC'] / 1e6,
                'TCI ($M)': r['TCI'] / 1e6,
                'OPEX ($M/yr)': r['opex_total'] / 1e6,
                'Revenue ($M/yr)': r['revenue_total'] / 1e6,
                'Net Income ($M/yr)': r['net_income'] / 1e6,
                'LCOA ($/t)': r['LCOA'],
                'NPV ($M)': r['NPV'] / 1e6,
                'IRR (%)': r['IRR'] * 100,
                'Payback (years)': r['PP'] if r['PP'] < 1000 else 'Infinite',
                'Viable': 'Yes' if r['viable'] else 'No',
            } for r in tech_results])
            tech_df.to_excel(writer, sheet_name='Technology Sensitivity', index=False)
        
        # CEPCI Data sheet
        cepci_df = pd.DataFrame({'Year': list(CEPCI_DATA.keys()), 'CEPCI': list(CEPCI_DATA.values())})
        cepci_df.to_excel(writer, sheet_name='CEPCI Data', index=False)
        
        # References sheet
        refs = pd.DataFrame({
            'Reference': [
                'Lei et al. (2024), Energy 307, 132728 - Economic methodology',
                'Turton R., et al. (2018). Analysis, Synthesis and Design of Chemical Processes, 5th Ed.',
                'Peters M.S., Timmerhaus K.D., West R.E. (2003). Plant Design and Economics, 5th Ed.',
                'DOE H2NEW Consortium (2024). NREL/TP-5900-87625 - PEM Electrolyzer costs',
                'Delikonstantis et al. (2017) - DBD Plasma reactor tubes',
                'Martini et al. (2019) - DBD Power supply costing',
                'Thunder Said Energy (2023) - Heat exchanger costs',
                'IMARC Group (2025) - Green ammonia and oxygen prices',
            ]
        })
        refs.to_excel(writer, sheet_name='References', index=False)
    
    print(f"\nResults saved to: {filename}")


# =============================================================================
# SECTION 14: MAIN FUNCTION
# =============================================================================

def main():
    """Main function - run complete analysis with technology sensitivity."""
    
    # Scenario 1: Nitrogen Purchase (no ASU capital)
    print("\n" + "="*80)
    print("SCENARIO 1: NITROGEN PURCHASE (no ASU capital)")
    print("="*80)
    
    results1 = run_complete_analysis(
        nitrogen_source='purchase',
        include_asu=False,
        verbose=True
    )
    
    # Scenario 2: On-site ASU (with ASU capital)
    print("\n" + "="*80)
    print("SCENARIO 2: ON-SITE ASU (with ASU capital)")
    print("="*80)
    
    results2 = run_complete_analysis(
        nitrogen_source='asu',
        include_asu=True,
        verbose=True
    )
    
    # Technology Sensitivity Analysis
    tech_results = run_technology_sensitivity(results1)
    
    # Generate Plots
    print("\n" + "="*80)
    print("GENERATING PLOTS")
    print("="*80)
    
    plot_cost_breakdown(results1['opex'], results1['revenue'], 'cost_revenue_breakdown.png')
    print("  Saved: cost_revenue_breakdown.png")
    
    plot_economic_indicators(results1, 'economic_indicators_complete.png')
    print("  Saved: economic_indicators_complete.png")
    
    plot_cashflow(results1['TCI'], results1['net_income'], 'cashflow_complete.png')
    print("  Saved: cashflow_complete.png")
    
    plot_material_balance('material_balance.png')
    print("  Saved: material_balance.png")
    
    plot_equipment_cost_breakdown(results1['equipment_results'], 'equipment_cost_breakdown.png')
    print("  Saved: equipment_cost_breakdown.png")
    
    plot_technology_sensitivity(tech_results, 'technology_sensitivity.png')
    
    # Save to Excel
    save_to_excel(results1, tech_results, 'Economic_Analysis_Integrated.xlsx')
    
    print("\n" + "="*80)
    print("ANALYSIS COMPLETE")
    print("="*80)
    print("\nGenerated files:")
    print("  - Economic_Analysis_Integrated.xlsx")
    print("  - cost_revenue_breakdown.png")
    print("  - economic_indicators_complete.png")
    print("  - cashflow_complete.png")
    print("  - material_balance.png")
    print("  - equipment_cost_breakdown.png")
    print("  - technology_sensitivity.png")
    
    return results1, results2, tech_results


# =============================================================================
# ENTRY POINT
# =============================================================================

if __name__ == "__main__":
    results1, results2, tech_results = main()







#!/usr/bin/env python3
"""
=============================================================================
INTEGRATED SENSITIVITY ANALYSIS - AMMONIA PLANT (PLASMA-BASED)
=============================================================================

This module provides comprehensive sensitivity analysis that is fully
integrated with the equipment costing functions from the main economic
analysis module.

Key improvements over the previous version:
1. Uses dynamic equipment cost calculations instead of hardcoded values
2. Consistent TCI and OPEX methodology with main economic analysis
3. Proper scaling of equipment costs using validated correlations
4. Integration with DBD plasma reactor thesis methodology

Sensitivities analyzed:
1. Steam flow rate (geothermal resource) - 13 cases
2. PEM electrolyzer capacity - 20 cases
3. Purge/recycle ratio - 19 cases
4. Kalina cycle NH3 purity - 7 cases

References:
- Lei et al. (2024), Energy 307, 132728
- Turton et al. (2018)
- DOE H2NEW Consortium (2024)
"""

import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from typing import Dict, List, Tuple

# =============================================================================
# SECTION 1: CEPCI AND COST ADJUSTMENT
# =============================================================================

CEPCI_DATA = {
    2003: 402.0, 2017: 567.5, 2020: 596.2, 2022: 816.0, 2025: 834.0
}
TARGET_YEAR = 2025
CEPCI_TARGET = CEPCI_DATA[TARGET_YEAR]


def cepci_adjust(cost_base: float, year_base: int = 2017) -> float:
    """Adjust cost to target year using CEPCI."""
    if year_base not in CEPCI_DATA:
        return cost_base
    return cost_base * (CEPCI_TARGET / CEPCI_DATA[year_base])


# =============================================================================
# SECTION 2: EQUIPMENT COST FUNCTIONS (from integrated analysis)
# =============================================================================

def calc_turbine(power_kW: float) -> Dict:
    """Turbine cost using Turton (2018) + Bejan (1996)."""
    power_kW = max(10, power_kW)
    power_hp = power_kW * 1.341
    
    K1, K2, K3 = 2.7051, 1.4398, -0.1776
    power_hp_calc = max(100, min(4000, power_hp))
    ln_P = np.log(power_hp_calc)
    Cp_turton = np.exp(K1 + K2 * ln_P + K3 * ln_P**2)
    
    if power_hp > 4000:
        Cp_turton = Cp_turton * (power_hp / 4000) ** 0.6
    
    Cp_turton_adj = cepci_adjust(Cp_turton, 2017)
    Cp_bejan = 6000 * (power_kW ** 0.7)
    Cp_bejan_adj = cepci_adjust(Cp_bejan, 1995)
    
    # Add 1995 to CEPCI if needed
    CEPCI_1995 = 381.1
    Cp_bejan_adj = Cp_bejan * (CEPCI_TARGET / CEPCI_1995)
    
    Cp_avg = 0.6 * Cp_turton_adj + 0.4 * Cp_bejan_adj
    F_BM = 3.5
    
    return {'purchased': Cp_avg, 'installed': Cp_avg * F_BM}


def calc_compressor(power_kW: float) -> Dict:
    """Compressor cost using Turton (2018)."""
    power_kW = max(5, power_kW)
    power_hp = power_kW * 1.341
    
    K1, K2, K3 = 2.2897, 1.3604, -0.1027
    power_hp_calc = max(450, min(3000, power_hp))
    ln_P = np.log(power_hp_calc)
    Cp = np.exp(K1 + K2 * ln_P + K3 * ln_P**2)
    
    if power_hp < 450:
        Cp = Cp * (power_hp / 450) ** 0.6
    elif power_hp > 3000:
        Cp = Cp * (power_hp / 3000) ** 0.6
    
    Cp_adj = max(50000, cepci_adjust(Cp, 2017))
    F_BM = 2.15
    
    return {'purchased': Cp_adj, 'installed': Cp_adj * F_BM}


def calc_pump(power_kW: float) -> Dict:
    """Pump cost using power law correlation."""
    power_kW = max(1, power_kW)
    Cp_2020 = max(5000, 1000 * (power_kW ** 0.70))
    Cp_adj = Cp_2020 * (CEPCI_TARGET / 596.2)
    F_BM = 2.4
    
    return {'purchased': Cp_adj, 'installed': Cp_adj * F_BM}


def calc_heat_exchanger(area_m2: float, hx_type: str = 'shell_tube') -> Dict:
    """Heat exchanger cost using industry power law."""
    if np.isnan(area_m2) or area_m2 <= 0:
        area_m2 = 10
    
    MAX_AREA = 1000
    base_cost_per_m2 = 350 if 'shell' in hx_type.lower() else 250
    scale_exp = 0.85
    
    n_shells = max(1, int(np.ceil(area_m2 / MAX_AREA)))
    area_per_shell = area_m2 / n_shells
    
    ref_area = 100
    Cp_ref = base_cost_per_m2 * ref_area
    Cp_per_shell = Cp_ref * (area_per_shell / ref_area) ** scale_exp
    Cp_total = Cp_per_shell * n_shells
    Cp_adj = Cp_total * (CEPCI_TARGET / 816.0)
    
    F_BM = 3.29
    
    return {'purchased': Cp_adj, 'installed': Cp_adj * F_BM}


def calc_vessel(volume_m3: float, diameter_m: float, P_barg: float = 10.0) -> Dict:
    """Pressure vessel cost using Peters-Timmerhaus."""
    height_m = volume_m3 / (np.pi * (diameter_m/2)**2)
    D_ft, L_ft = diameter_m * 3.281, height_m * 3.281
    D_in = diameter_m * 39.37
    P_psig = max(15, P_barg * 14.5038)
    
    S, E = 15000, 0.85
    t_shell = (P_psig * D_in) / (2 * S * E - 1.2 * P_psig)
    t_shell = max(0.25, min(3.0, t_shell))
    
    rho_steel = 490
    W_cyl = np.pi * D_ft * L_ft * (t_shell/12) * rho_steel
    W_heads = 2 * 0.25 * np.pi * (D_ft**2) * (t_shell/12) * rho_steel
    W_lb = max(1000, W_cyl + W_heads)
    
    Cp_2003 = 600 * (W_lb ** 0.65)
    Cp_adj = Cp_2003 * (CEPCI_TARGET / 402.0)
    
    Fp = 1.0 if P_barg <= 10 else min(2.0, 1.0 + 0.1 * (P_barg - 10) / 10)
    F_BM = 4.07
    
    return {'purchased': Cp_adj * Fp, 'installed': Cp_adj * Fp * F_BM}


def calc_pem_electrolyzer(power_kW: float) -> Dict:
    """PEM electrolyzer cost using DOE H2NEW (2024)."""
    power_kW = max(100, power_kW)
    cost_per_kW = 801  # $/kW (DOE 2024)
    Cp = power_kW * cost_per_kW
    F_BM = 1.12
    
    return {'purchased': Cp, 'installed': Cp * F_BM}


# DBD Plasma Reactor Reference Values
DBD_REFERENCE = {
    'n_tubes': 240,
    'power_kW': 114.7,
    'tube_cost_EUR': 15142.98,
    'power_supply_EUR': 173000,
    'tube_capacity_NLmin': 111,
    'cost_per_tube_effective': 63.10,
}


def calc_dbd_plasma_reactor(power_kW: float) -> Dict:
    """DBD plasma reactor cost using thesis methodology."""
    power_kW = max(10, power_kW)
    tube_capacity = DBD_REFERENCE['tube_capacity_NLmin']
    flow_NLmin = power_kW / 0.25
    n_tubes = max(1, int(np.ceil(flow_NLmin / tube_capacity)))
    
    # Tube bundle cost
    n_ref = DBD_REFERENCE['n_tubes']
    C_ref_tubes = DBD_REFERENCE['tube_cost_EUR']
    
    if n_tubes <= n_ref:
        C_tubes_EUR = DBD_REFERENCE['cost_per_tube_effective'] * n_tubes
    else:
        C_tubes_EUR = C_ref_tubes * (n_tubes / n_ref) ** 0.6
    
    # Power supply cost
    P_ref = DBD_REFERENCE['power_kW']
    C_ref_power = DBD_REFERENCE['power_supply_EUR']
    
    if power_kW <= 1000:
        C_power_EUR = 1500 * power_kW + 900
    else:
        C_power_EUR = C_ref_power * (power_kW / P_ref) ** 0.7
    
    C_total_EUR = C_tubes_EUR + C_power_EUR
    EUR_USD = 1.08
    Cp = C_total_EUR * EUR_USD
    F_BM = 1.8
    
    return {'purchased': Cp, 'installed': Cp * F_BM, 'n_tubes': n_tubes}


def calc_combustion_chamber(duty_MW: float) -> Dict:
    """Combustion chamber cost for purge gas."""
    duty_MW = max(0.5, duty_MW)
    
    burner_cost_per_MW = 4000 if duty_MW < 5 else (2500 if duty_MW < 20 else 1500)
    C_burner = max(10000, duty_MW * burner_cost_per_MW)
    
    volume_m3 = duty_MW / 10
    C_chamber = 40000 if volume_m3 < 2 else 30000 * (volume_m3 ** 0.6)
    C_controls = 15000 + duty_MW * 500
    C_stack = 10000 + duty_MW * 300
    
    Cp = max(75000, C_burner + C_chamber + C_controls + C_stack)
    F_BM = 2.5
    
    return {'purchased': Cp, 'installed': Cp * F_BM}


def calc_control_valve(Cv: float) -> Dict:
    """Control valve cost."""
    Cv = max(0.5, Cv)
    Cp = max(500, np.exp(3.7119 + 0.5974 * np.log(Cv)))
    Cp_adj = cepci_adjust(Cp, 2017)
    F_BM = 1.3
    
    return {'purchased': Cp_adj, 'installed': Cp_adj * F_BM}


# =============================================================================
# SECTION 3: ECONOMIC PARAMETERS
# =============================================================================

INTEREST_RATE = 0.14          # 14% discount rate
SYSTEM_LIFETIME = 20          # years
OPERATION_HOURS = 7300        # hours/year
NH3_LHV = 5.17               # kWh/kg (Lower Heating Value)

# Market Prices
PRICES = {
    'ammonia': 870.0,         # $/tonne green ammonia
    'oxygen': 150.0,          # $/tonne high-purity oxygen
    'electricity_sell': 0.10, # $/kWh
    'electricity_buy': 0.08,  # $/kWh
    'heating': 12.0,          # $/GJ district heating
    'nitrogen': 80.0,         # $/tonne
    'demin_water': 3.0,       # $/tonne
}

# Wind turbine parameters
WIND_CAPEX_PER_KW = 1800     # $/kW
WIND_CAPACITY_FACTOR = 0.35   # 35%

# Capital Recovery Factor
CRF = (INTEREST_RATE * (1 + INTEREST_RATE)**SYSTEM_LIFETIME) / \
      ((1 + INTEREST_RATE)**SYSTEM_LIFETIME - 1)

# Catalyst lifetimes and cost fractions
CATALYST_LIFETIMES = {'plasma_catalyst': 10, 'pem_stack': 7}
CATALYST_COST_FRACTIONS = {'plasma_catalyst': 0.15, 'pem_stack': 0.40}


# =============================================================================
# SECTION 4: BASE EQUIPMENT CONFIGURATION
# =============================================================================

# Base equipment sizing (from process simulation)
BASE_EQUIPMENT = {
    # Turbines
    'T-100': {'type': 'turbine', 'power_kW': 13472.97},
    'T-101': {'type': 'turbine', 'power_kW': 11167.13},
    'T-102': {'type': 'turbine', 'power_kW': 172.76},
    
    # Compressors
    'K-100': {'type': 'compressor', 'power_kW': 6.41},
    'K-101': {'type': 'compressor', 'power_kW': 8.70},
    'K-102': {'type': 'compressor', 'power_kW': 802.76},
    
    # Pumps
    'P-100': {'type': 'pump', 'power_kW': 15.67},
    'P-101': {'type': 'pump', 'power_kW': 347.94},
    
    # Heat Exchangers (total area ~2000 m²)
    'HX-total': {'type': 'heat_exchanger', 'area_m2': 1980},
    
    # Vessels (total ~20 m³)
    'V-total': {'type': 'vessel', 'volume_m3': 20, 'diameter_m': 1.5, 'P_barg': 50},
    
    # Control Valves
    'CV-total': {'type': 'control_valve', 'Cv': 200},
    
    # Special Equipment
    'DBD-100': {'type': 'dbd_plasma', 'power_kW': 12440},
    'PEM-100': {'type': 'pem_electrolyzer', 'power_kW': 5873},
    'CC-100': {'type': 'combustion_chamber', 'duty_MW': 2.5},
}


def calculate_base_equipment_costs() -> Dict:
    """Calculate base equipment costs using cost functions."""
    costs = {}
    total_pec = 0
    total_inst = 0
    
    for tag, params in BASE_EQUIPMENT.items():
        eq_type = params['type']
        
        if eq_type == 'turbine':
            result = calc_turbine(params['power_kW'])
        elif eq_type == 'compressor':
            result = calc_compressor(params['power_kW'])
        elif eq_type == 'pump':
            result = calc_pump(params['power_kW'])
        elif eq_type == 'heat_exchanger':
            result = calc_heat_exchanger(params['area_m2'])
        elif eq_type == 'vessel':
            result = calc_vessel(params['volume_m3'], params['diameter_m'], params['P_barg'])
        elif eq_type == 'control_valve':
            result = calc_control_valve(params['Cv'])
        elif eq_type == 'dbd_plasma':
            result = calc_dbd_plasma_reactor(params['power_kW'])
        elif eq_type == 'pem_electrolyzer':
            result = calc_pem_electrolyzer(params['power_kW'])
        elif eq_type == 'combustion_chamber':
            result = calc_combustion_chamber(params['duty_MW'])
        else:
            continue
        
        costs[tag] = result
        total_pec += result['purchased']
        total_inst += result['installed']
    
    return {
        'equipment': costs,
        'total_pec': total_pec,
        'total_inst': total_inst
    }


# =============================================================================
# SECTION 5: TCI AND OPEX CALCULATION
# =============================================================================

def calculate_TCI(installed_cost: float, include_asu: bool = False) -> Dict:
    """Calculate Total Capital Investment from installed cost."""
    ISBL = installed_cost
    
    if include_asu:
        asu_capacity_tpd = 1933 * 24 / 1000  # ~46 t/d
        asu_capital = asu_capacity_tpd * 250000
        ISBL += asu_capital * 0.5
    
    # Direct costs
    site_development = 0.05 * ISBL
    land = 0.02 * ISBL
    buildings = 0.08 * ISBL
    utility_systems = 0.10 * ISBL
    total_direct = ISBL + site_development + land + buildings + utility_systems
    
    # Indirect costs
    engineering = 0.12 * ISBL
    construction_mgmt = 0.08 * ISBL
    contingencies = 0.18 * ISBL
    total_indirect = engineering + construction_mgmt + contingencies
    
    FCI = total_direct + total_indirect
    startup_costs = 0.08 * FCI
    TCI = (FCI + startup_costs) / 0.85
    working_capital = 0.15 * TCI
    
    return {
        'ISBL': ISBL,
        'FCI': FCI,
        'TCI': TCI,
        'working_capital': working_capital,
    }


def calculate_OPEX(FCI: float, plasma_purchased: float, pem_purchased: float,
                   n2_tonnes_year: float = 14111, water_tonnes_year: float = 6575) -> Dict:
    """Calculate annual operating costs."""
    opex = {}
    
    # Raw materials
    opex['nitrogen'] = n2_tonnes_year * PRICES['nitrogen']
    opex['demin_water'] = water_tonnes_year * PRICES['demin_water']
    
    # Utilities (cooling water)
    cooling_duty_kW = 76355  # From process
    cooling_GJ_year = cooling_duty_kW * 3.6 * OPERATION_HOURS / 1e6
    opex['cooling_water'] = cooling_GJ_year * 0.354
    
    # Fixed costs
    opex['maintenance'] = 0.025 * FCI
    opex['labor'] = 4 * 75000
    opex['insurance_taxes'] = 0.015 * FCI
    
    # Catalyst/electrode replacement
    plasma_catalyst = plasma_purchased * CATALYST_COST_FRACTIONS['plasma_catalyst']
    plasma_annual = plasma_catalyst / CATALYST_LIFETIMES['plasma_catalyst']
    
    pem_stack = pem_purchased * CATALYST_COST_FRACTIONS['pem_stack']
    pem_annual = pem_stack / CATALYST_LIFETIMES['pem_stack']
    
    opex['catalyst_replacement'] = plasma_annual + pem_annual
    opex['total'] = sum(opex.values())
    
    return opex


# =============================================================================
# SECTION 6: LPC CALCULATION
# =============================================================================

def calculate_LPC(TCI: float, OPEX: float, nh3_kg_s: float, net_power_kW: float,
                  d_heat_kW: float, o2_tonnes_year: float = 5840,
                  include_wind: bool = False, wind_cap_kW: float = 0) -> Dict:
    """
    Calculate Levelized Production Cost using three methods.
    
    Methods:
    - LPC_nh3: Conservative (only NH3 energy in denominator)
    - LPC_total: All energy outputs in denominator
    - LPC_net: Net cost after byproduct credits (recommended)
    
    Parameters:
    -----------
    TCI : float
        Total Capital Investment ($)
    OPEX : float
        Annual operating expenses ($/year)
    nh3_kg_s : float
        Ammonia production rate (kg/s)
    net_power_kW : float
        Net power output (positive=export, negative=import)
    d_heat_kW : float
        District heating output (kW thermal)
    o2_tonnes_year : float
        Oxygen byproduct production (tonnes/year)
    include_wind : bool
        Include wind turbine CAPEX for negative power cases
    wind_cap_kW : float
        Wind turbine capacity if include_wind=True
    """
    # Energy outputs (kWh/year)
    nh3_tpy = nh3_kg_s * 3600 * OPERATION_HOURS / 1000
    E_nh3 = nh3_tpy * 1000 * NH3_LHV
    E_elec = max(0, net_power_kW) * OPERATION_HOURS
    E_heat = d_heat_kW * OPERATION_HOURS
    E_total = E_nh3 + E_elec + E_heat
    
    # Adjust TCI for wind turbine if needed
    if include_wind and net_power_kW < 0 and wind_cap_kW > 0:
        wind_capex = wind_cap_kW * WIND_CAPEX_PER_KW
        TCI_adj = TCI + wind_capex
    else:
        TCI_adj = TCI
        wind_capex = 0
    
    # Annualized capital cost
    ann_capex = TCI_adj * CRF
    
    # Adjust OPEX for electricity purchase if net power is negative and no wind
    if net_power_kW < 0 and not include_wind:
        elec_purchase = abs(net_power_kW) * OPERATION_HOURS * PRICES['electricity_buy']
        adj_opex = OPEX + elec_purchase
    else:
        adj_opex = OPEX
    
    total_annual_cost = ann_capex + adj_opex
    
    # Revenue from products
    rev_nh3 = nh3_tpy * PRICES['ammonia']
    rev_o2 = o2_tonnes_year * PRICES['oxygen']
    rev_elec = E_elec * PRICES['electricity_sell']
    rev_heat = d_heat_kW * OPERATION_HOURS * 0.0036 * PRICES['heating']
    
    total_revenue = rev_nh3 + rev_o2 + rev_elec + rev_heat
    byproduct_revenue = rev_o2 + rev_elec + rev_heat  # Exclude NH3
    
    # Three LPC approaches
    LPC_nh3_only = total_annual_cost / E_nh3 if E_nh3 > 0 else np.inf
    LPC_total = total_annual_cost / E_total if E_total > 0 else np.inf
    LPC_net = (total_annual_cost - byproduct_revenue) / E_nh3 if E_nh3 > 0 else np.inf
    
    # Additional economic indicators
    net_income = total_revenue - adj_opex
    PVAF = (1 - (1 + INTEREST_RATE)**(-SYSTEM_LIFETIME)) / INTEREST_RATE
    NPV = -TCI_adj + net_income * PVAF
    PBP = TCI_adj / net_income if net_income > 0 else np.inf
    
    # LCOA ($/tonne)
    LCOA = total_annual_cost / nh3_tpy if nh3_tpy > 0 else np.inf
    LCOA_net = (total_annual_cost - byproduct_revenue) / nh3_tpy if nh3_tpy > 0 else np.inf
    
    return {
        'NH3_tpy': nh3_tpy,
        'E_nh3_GWh': E_nh3 / 1e6,
        'E_elec_GWh': E_elec / 1e6,
        'E_heat_GWh': E_heat / 1e6,
        'E_total_GWh': E_total / 1e6,
        'TCI': TCI_adj,
        'Wind_CAPEX': wind_capex,
        'OPEX': adj_opex,
        'Ann_Cost': total_annual_cost,
        'Rev_NH3': rev_nh3,
        'Rev_O2': rev_o2,
        'Rev_Elec': rev_elec,
        'Rev_Heat': rev_heat,
        'Rev_Total': total_revenue,
        'Rev_Byproduct': byproduct_revenue,
        'Net_Income': net_income,
        'LPC_nh3': LPC_nh3_only,
        'LPC_total': LPC_total,
        'LPC_net': LPC_net,
        'LCOA': LCOA,
        'LCOA_net': LCOA_net,
        'NPV': NPV,
        'PBP': PBP,
    }


# =============================================================================
# SECTION 7: SENSITIVITY 1 - STEAM FLOW (GEOTHERMAL)
# =============================================================================

def run_steam_sensitivity() -> pd.DataFrame:
    """
    Steam flow sensitivity analysis.
    
    Varies geothermal steam flow rate, which affects:
    - Turbine power output (and thus equipment costs)
    - Net power balance
    - Potential need for wind turbines at low steam rates
    """
    print("\n" + "=" * 100)
    print("SENSITIVITY 1: STEAM FLOW RATE (Geothermal Resource)")
    print("=" * 100)
    
    # Constants for this sensitivity
    NH3_KG_S = 0.1066
    D_HEAT_KW = 53000
    O2_TPY = 5840
    
    # Steam flow data: (Case, Steam kg/s, Net Power kW, Wind Need, W1 kW, W2 kW)
    steam_data = [
        (1, 100, -11100, True, 4491, 3722),
        (2, 120, -9481, False, 5389, 4467),
        (3, 140, -7863, False, 6287, 5211),
        (4, 160, -6244, False, 7186, 5956),
        (5, 180, -4626, False, 8084, 6700),
        (6, 200, -3007, False, 8982, 7445),
        (7, 220, -1389, False, 9880, 8189),
        (8, 240, 229.5, False, 10780, 8934),  # Base case
        (9, 260, 1848, False, 11680, 9678),
        (10, 280, 3466, False, 12570, 10420),
        (11, 300, 5085, False, 13470, 11170),
        (12, 320, 6703, False, 14370, 11910),
        (13, 340, 8322, False, 15270, 12660),
    ]
    
    # Calculate base equipment costs (excluding turbines)
    base_costs = calculate_base_equipment_costs()
    turbine_tags = ['T-100', 'T-101', 'T-102']
    base_turbine_inst = sum(base_costs['equipment'][t]['installed'] for t in turbine_tags)
    base_inst_no_turbine = base_costs['total_inst'] - base_turbine_inst
    
    plasma_purchased = base_costs['equipment']['DBD-100']['purchased']
    pem_purchased = base_costs['equipment']['PEM-100']['purchased']
    
    results = []
    for case, steam, pwr, wind_need, w1, w2 in steam_data:
        # Calculate turbine costs based on actual power output
        t1_cost = calc_turbine(w1)
        t2_cost = calc_turbine(w2)
        t3_cost = calc_turbine(172.76)  # T-102 is constant (syngas expander)
        
        turbine_inst = t1_cost['installed'] + t2_cost['installed'] + t3_cost['installed']
        total_inst = base_inst_no_turbine + turbine_inst
        
        # Calculate TCI and OPEX
        capital = calculate_TCI(total_inst, include_asu=False)
        opex = calculate_OPEX(capital['FCI'], plasma_purchased, pem_purchased)
        
        # Wind capacity for negative power cases
        wind_cap = abs(pwr) / WIND_CAPACITY_FACTOR if pwr < 0 else 0
        
        # Calculate LPC
        lpc = calculate_LPC(
            capital['TCI'], opex['total'], NH3_KG_S, pwr, D_HEAT_KW, O2_TPY,
            include_wind=wind_need, wind_cap_kW=wind_cap
        )
        
        results.append({
            'Case': case,
            'Steam_kg_s': steam,
            'W1_kW': w1,
            'W2_kW': w2,
            'Turbine_Inst': turbine_inst,
            'Net_Power_kW': pwr,
            'Wind_Need': 'YES' if wind_need else 'NO',
            'Wind_Cap_kW': wind_cap,
            'ISBL': total_inst,
            **lpc
        })
    
    df = pd.DataFrame(results)
    
    # Print summary
    print(f"\n{'Case':<6} {'Steam':<8} {'Net kW':<10} {'Wind':<6} {'TCI $M':<10} "
          f"{'LPC_NH3':<12} {'LPC_Net':<12} {'NPV $M':<12}")
    print("-" * 100)
    for _, row in df.iterrows():
        print(f"{row['Case']:<6} {row['Steam_kg_s']:<8.0f} {row['Net_Power_kW']:<10.0f} "
              f"{row['Wind_Need']:<6} {row['TCI']/1e6:<10.2f} "
              f"${row['LPC_nh3']:<11.4f} ${row['LPC_net']:<11.4f} "
              f"{row['NPV']/1e6:<12.2f}")
    
    return df


# =============================================================================
# SECTION 8: SENSITIVITY 2 - PEM ELECTROLYZER CAPACITY
# =============================================================================

def run_pem_sensitivity() -> pd.DataFrame:
    """
    PEM electrolyzer capacity sensitivity analysis.
    
    Varies PEM capacity, which affects:
    - Equipment cost (PEM scales linearly with power at $801/kW)
    - Hydrogen production rate
    - Ammonia production rate
    - Net power consumption
    """
    print("\n" + "=" * 100)
    print("SENSITIVITY 2: PEM ELECTROLYZER CAPACITY")
    print("=" * 100)
    
    # PEM data: (Case, PEM_kW, NH3_kg/s, D_Heat_kW, Net_Power_kW)
    pem_data = [
        (1, 4885, 0.1081, 52900, 6370),
        (2, 5142, 0.1078, 53150, 6100),
        (3, 5399, 0.1075, 53400, 5829),
        (4, 5657, 0.1071, 53650, 5559),
        (5, 5914, 0.1068, 53910, 5289),
        (6, 6171, 0.1064, 54170, 5018),
        (7, 6428, 0.1061, 54450, 4748),
        (8, 6685, 0.1058, 54730, 4478),
        (9, 6942, 0.1054, 55020, 4208),
        (10, 7199, 0.1051, 55300, 3937),
        (11, 7456, 0.1048, 55570, 3667),
        (12, 7714, 0.1044, 55830, 3397),
        (13, 7971, 0.1041, 56080, 3126),
        (14, 8228, 0.1038, 56320, 2856),
        (15, 8485, 0.1034, 56560, 2586),
        (16, 8742, 0.1031, 56800, 2316),
        (17, 8999, 0.1028, 57020, 2046),
        (18, 9256, 0.1024, 56150, 1777),
        (19, 9513, 0.1021, 56250, 1507),
        (20, 9771, 0.1018, 56350, 1236),
    ]
    
    # Calculate base equipment costs (excluding PEM)
    base_costs = calculate_base_equipment_costs()
    pem_base_inst = base_costs['equipment']['PEM-100']['installed']
    base_inst_no_pem = base_costs['total_inst'] - pem_base_inst
    
    plasma_purchased = base_costs['equipment']['DBD-100']['purchased']
    
    results = []
    for case, pem_kw, nh3, dheat, pwr in pem_data:
        # Calculate PEM cost at this capacity
        pem_cost = calc_pem_electrolyzer(pem_kw)
        total_inst = base_inst_no_pem + pem_cost['installed']
        
        # Scale O2 production with PEM capacity
        o2_tpy = 5840 * (pem_kw / 5873)  # Scale from base case
        
        # Calculate TCI and OPEX
        capital = calculate_TCI(total_inst, include_asu=False)
        opex = calculate_OPEX(capital['FCI'], plasma_purchased, pem_cost['purchased'])
        
        # Calculate LPC
        lpc = calculate_LPC(
            capital['TCI'], opex['total'], nh3, pwr, dheat, o2_tpy
        )
        
        results.append({
            'Case': case,
            'PEM_kW': pem_kw,
            'PEM_Cost': pem_cost['installed'],
            'NH3_kg_s': nh3,
            'D_Heat_kW': dheat,
            'O2_tpy': o2_tpy,
            'Net_Power_kW': pwr,
            'ISBL': total_inst,
            **lpc
        })
    
    df = pd.DataFrame(results)
    
    # Print summary
    print(f"\n{'Case':<6} {'PEM kW':<8} {'NH3 kg/s':<10} {'Net kW':<10} "
          f"{'LPC_NH3':<12} {'LPC_Net':<12} {'LCOA_Net':<12}")
    print("-" * 90)
    for _, row in df.iterrows():
        print(f"{row['Case']:<6} {row['PEM_kW']:<8.0f} {row['NH3_kg_s']:<10.4f} "
              f"{row['Net_Power_kW']:<10.0f} "
              f"${row['LPC_nh3']:<11.4f} ${row['LPC_net']:<11.4f} "
              f"${row['LCOA_net']:<11.0f}")
    
    return df


# =============================================================================
# SECTION 9: SENSITIVITY 3 - PURGE/RECYCLE RATIO
# =============================================================================

def run_purge_sensitivity() -> pd.DataFrame:
    """
    Purge/Recycle ratio sensitivity analysis.
    
    Varies the purge fraction, which affects:
    - Ammonia production rate (higher purge = lower production)
    - Plasma reactor power consumption
    - Net power balance
    - Compressor power (lower recycle = smaller compressors)
    """
    print("\n" + "=" * 100)
    print("SENSITIVITY 3: PURGE/RECYCLE RATIO")
    print("=" * 100)
    
    KALINA_HEAT = 53000  # kW from Kalina cycle (constant)
    
    # Purge data: (Case, Purge%, NH3_kg/s, Plasma_Heat_kW, Plasma_kW, Net_Power_kW)
    purge_data = [
        (1, 0.14, 0.1531, 43.89, 17700, -900),
        (2, 0.1853, 0.1487, 129.4, 18740, -1708),
        (3, 0.2305, 0.1486, 141.6, 17870, -679),
        (4, 0.2758, 0.1485, 153.7, 17280, -10.7),
        (5, 0.3211, 0.1483, 165.8, 16860, 494.4),
        (6, 0.3663, 0.1421, 301.7, 16050, 1356),
        (7, 0.4116, 0.1280, 590.8, 14640, 2802),
        (8, 0.4568, 0.1162, 835.8, 13440, 4047),
        (9, 0.5021, 0.1061, 1048, 12380, 5134),
        (10, 0.5474, 0.09742, 1227, 11470, 6085),
        (11, 0.5926, 0.08980, 1413, 10650, 6922),
        (12, 0.6379, 0.08309, 1576, 9899, 7668),
        (13, 0.6832, 0.07715, 1722, 9210, 8335),
        (14, 0.7284, 0.07188, 1850, 8577, 8933),
        (15, 0.7737, 0.06720, 1966, 7991, 9476),
        (16, 0.8189, 0.06301, 2071, 7448, 9967),
        (17, 0.8642, 0.05924, 2165, 7291, 10410),
        (18, 0.9095, 0.05573, 2251, 6896, 10820),
        (19, 0.9547, 0.05251, 2329, 6534, 11200),
    ]
    
    # Calculate base equipment costs (excluding plasma reactor)
    base_costs = calculate_base_equipment_costs()
    plasma_base_inst = base_costs['equipment']['DBD-100']['installed']
    base_inst_no_plasma = base_costs['total_inst'] - plasma_base_inst
    
    pem_purchased = base_costs['equipment']['PEM-100']['purchased']
    
    results = []
    for case, purge, nh3, plasma_heat, plasma_kw, pwr in purge_data:
        # Calculate plasma reactor cost at this power level
        plasma_cost = calc_dbd_plasma_reactor(plasma_kw)
        total_inst = base_inst_no_plasma + plasma_cost['installed']
        
        # Total district heating = Kalina + plasma exchanger
        total_dheat = KALINA_HEAT + plasma_heat
        
        # Scale O2 and N2 with ammonia production
        scale_factor = nh3 / 0.1066
        o2_tpy = 5840 * scale_factor
        n2_tpy = 14111 * scale_factor
        
        # Calculate TCI and OPEX
        capital = calculate_TCI(total_inst, include_asu=False)
        opex = calculate_OPEX(capital['FCI'], plasma_cost['purchased'], pem_purchased,
                              n2_tonnes_year=n2_tpy)
        
        # Calculate LPC
        lpc = calculate_LPC(
            capital['TCI'], opex['total'], nh3, pwr, total_dheat, o2_tpy
        )
        
        results.append({
            'Case': case,
            'Purge_%': purge * 100,
            'Recycle_%': (1 - purge) * 100,
            'NH3_kg_s': nh3,
            'Plasma_kW': plasma_kw,
            'Plasma_Cost': plasma_cost['installed'],
            'D_Heat_kW': total_dheat,
            'Net_Power_kW': pwr,
            'ISBL': total_inst,
            **lpc
        })
    
    df = pd.DataFrame(results)
    
    # Print summary
    print(f"\n{'Case':<6} {'Purge%':<8} {'NH3 kg/s':<10} {'Plasma kW':<10} {'Net kW':<10} "
          f"{'LPC_NH3':<12} {'LPC_Net':<12}")
    print("-" * 90)
    for _, row in df.iterrows():
        print(f"{row['Case']:<6} {row['Purge_%']:<8.1f} {row['NH3_kg_s']:<10.4f} "
              f"{row['Plasma_kW']:<10.0f} {row['Net_Power_kW']:<10.0f} "
              f"${row['LPC_nh3']:<11.4f} ${row['LPC_net']:<11.4f}")
    
    return df


# =============================================================================
# SECTION 10: SENSITIVITY 4 - KALINA NH3 PURITY
# =============================================================================

def run_kalina_sensitivity() -> pd.DataFrame:
    """
    Kalina cycle NH3 purity sensitivity analysis.
    
    Varies the ammonia mass fraction in the Kalina working fluid, which affects:
    - Kalina cycle power output
    - District heating capacity
    - Overall system energy balance
    """
    print("\n" + "=" * 100)
    print("SENSITIVITY 4: KALINA CYCLE NH3 PURITY")
    print("=" * 100)
    
    NH3_KG_S = 0.1066
    O2_TPY = 5840
    
    # Kalina data: (Case, NH3_Frac, Kalina_Power_kW, Kalina_District_kW, Net_Power_kW, TCI)
    # Note: TCI from original Excel includes all equipment at that configuration
    kalina_data = [
        (1, 0.3, 0, 0, -6074, 188442739),
        (2, 0.4, 3718, 29460, -2333, 159259722),
        (3, 0.5, 7459, 53360, 1398, 141672844),
        (4, 0.6, 11160, 49980, 5086, 147382366),
        (5, 0.7, 14820, 46630, 8732, 152433599),
        (6, 0.8, 18430, 43320, 12340, 156806110),
        (7, 0.9, 21120, 32910, 15010, 158888642),
    ]
    
    # Get base equipment costs for plasma and PEM
    base_costs = calculate_base_equipment_costs()
    plasma_purchased = base_costs['equipment']['DBD-100']['purchased']
    pem_purchased = base_costs['equipment']['PEM-100']['purchased']
    
    results = []
    for case, nh3_frac, kal_pwr, kal_heat, pwr, tci in kalina_data:
        # Use provided TCI (complex relationship with Kalina configuration)
        # Calculate OPEX based on estimated FCI
        fci_estimate = tci * 0.85 / 1.08  # Reverse TCI calculation
        opex = calculate_OPEX(fci_estimate, plasma_purchased, pem_purchased)
        
        # Wind capacity for negative power cases
        wind_need = pwr < 0
        wind_cap = abs(pwr) / WIND_CAPACITY_FACTOR if wind_need else 0
        
        # Calculate LPC
        lpc = calculate_LPC(
            tci, opex['total'], NH3_KG_S, pwr, kal_heat, O2_TPY,
            include_wind=wind_need, wind_cap_kW=wind_cap
        )
        
        results.append({
            'Case': case,
            'NH3_Fraction': nh3_frac,
            'NH3_Fraction_%': nh3_frac * 100,
            'Kalina_Power_kW': kal_pwr,
            'Kalina_District_kW': kal_heat,
            'Net_Power_kW': pwr,
            'Wind_Need': 'YES' if wind_need else 'NO',
            'Wind_Cap_kW': wind_cap,
            **lpc
        })
    
    df = pd.DataFrame(results)
    
    # Print summary
    print(f"\n{'Case':<6} {'NH3%':<6} {'Kalina kW':<10} {'D.Heat kW':<12} {'Net kW':<10} "
          f"{'LPC_NH3':<12} {'LPC_Net':<12} {'NPV $M':<12}")
    print("-" * 100)
    for _, row in df.iterrows():
        print(f"{row['Case']:<6} {row['NH3_Fraction_%']:<6.0f} {row['Kalina_Power_kW']:<10.0f} "
              f"{row['Kalina_District_kW']:<12.0f} {row['Net_Power_kW']:<10.0f} "
              f"${row['LPC_nh3']:<11.4f} ${row['LPC_net']:<11.4f} "
              f"{row['NPV']/1e6:<12.2f}")
    
    return df


# =============================================================================
# SECTION 11: PLOTTING FUNCTIONS
# =============================================================================

def plot_sensitivity_comparison(steam_df: pd.DataFrame, pem_df: pd.DataFrame,
                                 purge_df: pd.DataFrame, kalina_df: pd.DataFrame,
                                 save_path: str):
    """Create comparison plots for all sensitivities."""
    fig, axes = plt.subplots(2, 2, figsize=(14, 12))
    
    # Steam sensitivity
    ax = axes[0, 0]
    ax.plot(steam_df['Steam_kg_s'], steam_df['LPC_nh3'], 'b-o', label='LPC_NH3', linewidth=2)
    ax.plot(steam_df['Steam_kg_s'], steam_df['LPC_net'], 'g-s', label='LPC_Net', linewidth=2)
    ax.set_xlabel('Steam Flow Rate (kg/s)', fontsize=11)
    ax.set_ylabel('LPC ($/kWh)', fontsize=11)
    ax.set_title('Steam Flow Sensitivity', fontsize=12, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.axhline(0, color='black', linewidth=0.5)
    
    # PEM sensitivity
    ax = axes[0, 1]
    ax.plot(pem_df['PEM_kW'], pem_df['LPC_nh3'], 'b-o', label='LPC_NH3', linewidth=2)
    ax.plot(pem_df['PEM_kW'], pem_df['LPC_net'], 'g-s', label='LPC_Net', linewidth=2)
    ax.set_xlabel('PEM Electrolyzer Capacity (kW)', fontsize=11)
    ax.set_ylabel('LPC ($/kWh)', fontsize=11)
    ax.set_title('PEM Capacity Sensitivity', fontsize=12, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Purge sensitivity
    ax = axes[1, 0]
    ax.plot(purge_df['Purge_%'], purge_df['LPC_nh3'], 'b-o', label='LPC_NH3', linewidth=2)
    ax.plot(purge_df['Purge_%'], purge_df['LPC_net'], 'g-s', label='LPC_Net', linewidth=2)
    ax.set_xlabel('Purge Fraction (%)', fontsize=11)
    ax.set_ylabel('LPC ($/kWh)', fontsize=11)
    ax.set_title('Purge/Recycle Ratio Sensitivity', fontsize=12, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Kalina sensitivity
    ax = axes[1, 1]
    ax.plot(kalina_df['NH3_Fraction_%'], kalina_df['LPC_nh3'], 'b-o', label='LPC_NH3', linewidth=2)
    ax.plot(kalina_df['NH3_Fraction_%'], kalina_df['LPC_net'], 'g-s', label='LPC_Net', linewidth=2)
    ax.set_xlabel('Kalina NH3 Purity (%)', fontsize=11)
    ax.set_ylabel('LPC ($/kWh)', fontsize=11)
    ax.set_title('Kalina NH3 Purity Sensitivity', fontsize=12, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"  Saved: {save_path}")


def plot_npv_comparison(steam_df: pd.DataFrame, pem_df: pd.DataFrame,
                        purge_df: pd.DataFrame, kalina_df: pd.DataFrame,
                        save_path: str):
    """Create NPV comparison plots for all sensitivities."""
    fig, axes = plt.subplots(2, 2, figsize=(14, 12))
    
    # Steam sensitivity NPV
    ax = axes[0, 0]
    npv_vals = steam_df['NPV'] / 1e6
    colors = ['forestgreen' if n > 0 else 'crimson' for n in npv_vals]
    ax.bar(steam_df['Steam_kg_s'], npv_vals, color=colors, edgecolor='black')
    ax.axhline(0, color='black', linewidth=1)
    ax.set_xlabel('Steam Flow Rate (kg/s)', fontsize=11)
    ax.set_ylabel('NPV (Million $)', fontsize=11)
    ax.set_title('Steam Flow Sensitivity - NPV', fontsize=12, fontweight='bold')
    ax.grid(axis='y', alpha=0.3)
    
    # PEM sensitivity NPV
    ax = axes[0, 1]
    npv_vals = pem_df['NPV'] / 1e6
    colors = ['forestgreen' if n > 0 else 'crimson' for n in npv_vals]
    ax.bar(range(len(pem_df)), npv_vals, color=colors, edgecolor='black')
    ax.axhline(0, color='black', linewidth=1)
    ax.set_xlabel('Case Number', fontsize=11)
    ax.set_ylabel('NPV (Million $)', fontsize=11)
    ax.set_title('PEM Capacity Sensitivity - NPV', fontsize=12, fontweight='bold')
    ax.grid(axis='y', alpha=0.3)
    
    # Purge sensitivity NPV
    ax = axes[1, 0]
    npv_vals = purge_df['NPV'] / 1e6
    colors = ['forestgreen' if n > 0 else 'crimson' for n in npv_vals]
    ax.bar(purge_df['Purge_%'], npv_vals, color=colors, edgecolor='black', width=3)
    ax.axhline(0, color='black', linewidth=1)
    ax.set_xlabel('Purge Fraction (%)', fontsize=11)
    ax.set_ylabel('NPV (Million $)', fontsize=11)
    ax.set_title('Purge Ratio Sensitivity - NPV', fontsize=12, fontweight='bold')
    ax.grid(axis='y', alpha=0.3)
    
    # Kalina sensitivity NPV
    ax = axes[1, 1]
    npv_vals = kalina_df['NPV'] / 1e6
    colors = ['forestgreen' if n > 0 else 'crimson' for n in npv_vals]
    ax.bar(kalina_df['NH3_Fraction_%'], npv_vals, color=colors, edgecolor='black', width=5)
    ax.axhline(0, color='black', linewidth=1)
    ax.set_xlabel('Kalina NH3 Purity (%)', fontsize=11)
    ax.set_ylabel('NPV (Million $)', fontsize=11)
    ax.set_title('Kalina Purity Sensitivity - NPV', fontsize=12, fontweight='bold')
    ax.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"  Saved: {save_path}")


# =============================================================================
# SECTION 12: MAIN EXECUTION
# =============================================================================

def main():
    """Run all sensitivity analyses and generate outputs."""
    print("=" * 100)
    print("INTEGRATED SENSITIVITY ANALYSIS")
    print("Using Dynamic Equipment Cost Functions")
    print("=" * 100)
    
    print(f"\nCapital Recovery Factor (CRF): {CRF:.4f} ({CRF*100:.2f}%)")
    print(f"Interest Rate: {INTEREST_RATE*100:.0f}%")
    print(f"System Lifetime: {SYSTEM_LIFETIME} years")
    print(f"Operation Hours: {OPERATION_HOURS} h/year")
    
    # Calculate and display base equipment costs
    print("\n" + "=" * 80)
    print("BASE EQUIPMENT COSTS (Dynamic Calculation)")
    print("=" * 80)
    base_costs = calculate_base_equipment_costs()
    print(f"  Total Purchased Equipment Cost: ${base_costs['total_pec']:,.0f}")
    print(f"  Total Installed Equipment Cost: ${base_costs['total_inst']:,.0f}")
    
    # Run all sensitivities
    steam_df = run_steam_sensitivity()
    pem_df = run_pem_sensitivity()
    purge_df = run_purge_sensitivity()
    kalina_df = run_kalina_sensitivity()
    
    # Generate plots
    print("\n" + "=" * 80)
    print("GENERATING PLOTS")
    print("=" * 80)
    
    plot_sensitivity_comparison(steam_df, pem_df, purge_df, kalina_df,
                                'sensitivity_lpc_comparison.png')
    plot_npv_comparison(steam_df, pem_df, purge_df, kalina_df,
                        'sensitivity_npv_comparison.png')
    
    # Save to Excel
    output_file = 'Sensitivity_Analysis_Integrated.xlsx'
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        steam_df.to_excel(writer, sheet_name='Steam_Sensitivity', index=False)
        pem_df.to_excel(writer, sheet_name='PEM_Sensitivity', index=False)
        purge_df.to_excel(writer, sheet_name='Purge_Sensitivity', index=False)
        kalina_df.to_excel(writer, sheet_name='Kalina_Sensitivity', index=False)
        
        # Summary sheet
        summary_data = []
        for name, df, param in [
            ("Steam Flow", steam_df, 'Steam_kg_s'),
            ("PEM Size", pem_df, 'PEM_kW'),
            ("Purge Ratio", purge_df, 'Purge_%'),
            ("Kalina NH3", kalina_df, 'NH3_Fraction'),
        ]:
            best_idx = df['LPC_net'].idxmin()
            best = df.loc[best_idx]
            summary_data.append({
                'Sensitivity': name,
                'Best Case': best['Case'],
                'Parameter Value': best[param],
                'LPC_NH3 ($/kWh)': best['LPC_nh3'],
                'LPC_Net ($/kWh)': best['LPC_net'],
                'NPV ($M)': best['NPV'] / 1e6,
                'TCI ($M)': best['TCI'] / 1e6,
            })
        
        summary_df = pd.DataFrame(summary_data)
        summary_df.to_excel(writer, sheet_name='Summary', index=False)
    
    print(f"\n  Saved: {output_file}")
    
    # Print optimal case summary
    print("\n" + "=" * 100)
    print("OPTIMAL CASE SUMMARY (Using LPC_Net - Recommended)")
    print("=" * 100)
    
    print(f"\n{'Sensitivity':<15} {'Best Case':<12} {'Parameter':<15} {'LPC_Net $/kWh':<15} {'NPV $M':<12}")
    print("-" * 70)
    
    for row in summary_data:
        param_val = row['Parameter Value']
        if row['Sensitivity'] == "Kalina NH3":
            param_str = f"{param_val*100:.0f}%"
        elif row['Sensitivity'] == "Purge Ratio":
            param_str = f"{param_val:.1f}%"
        else:
            param_str = f"{param_val:.0f}"
        
        print(f"{row['Sensitivity']:<15} Case {row['Best Case']:<6.0f} {param_str:<15} "
              f"${row['LPC_Net ($/kWh)']:<14.4f} {row['NPV ($M)']:<12.1f}")
    
    print("\n" + "=" * 100)
    print("ANALYSIS COMPLETE")
    print("=" * 100)
    print("\nGenerated files:")
    print("  - Sensitivity_Analysis_Integrated.xlsx")
    print("  - sensitivity_lpc_comparison.png")
    print("  - sensitivity_npv_comparison.png")
    
    return steam_df, pem_df, purge_df, kalina_df


# =============================================================================
# ENTRY POINT
# =============================================================================

if __name__ == "__main__":
    steam_df, pem_df, purge_df, kalina_df = main()



SCENARIO 1: NITROGEN PURCHASE (no ASU capital)
COMPLETE ECONOMIC ANALYSIS - PLASMA AMMONIA WITH ALL MATERIAL INPUTS

*** EQUIPMENT COSTS CALCULATED DYNAMICALLY USING VALIDATED CORRELATIONS ***


EQUIPMENT COST CALCULATION (DYNAMIC USING COST FUNCTIONS)
Target Year: 2025, CEPCI: 834.0

[EQUIPMENT COST SUMMARY BY TYPE]
----------------------------------------------------------------------
  turbine                  : $   7,856,623 / $  27,498,179 (3 items, 53.0%)
  dbd_plasma               : $   4,991,451 / $   8,984,612 (1 items, 17.3%)
  vessel                   : $   1,795,957 / $   7,309,543 (4 items, 14.1%)
  pem_electrolyzer         : $   4,704,273 / $   5,268,786 (1 items, 10.2%)
  heat_exchanger           : $     566,383 / $   1,980,512 (8 items, 3.8%)
  pump                     : $     135,760 / $     325,825 (2 items, 0.6%)
  compressor               : $     150,000 / $     322,500 (3 items, 0.6%)
  combustion_chamber       : $      77,000 / $     192,500 (1 items, 0.4%)
  con